In [256]:
import json
from pandas import json_normalize
import requests
import pandas as pd


# Testing the "generate_function_from_openapi" function

**Purpose**

This notebook performs several tests to determine if the function `generate_function_from_openapi` is performing as expected.

The function returns both data and metadata.

**Method**

20 End points from Elexon's Insights API were randomly drawn for testing.

1. Data is extracted from the `generate_function_from_openapi` function to verify
that it returns a response.

2. Data is manually retrieved using the "try it now" feature on the API website ([Elexon Insights API](https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-generation-availability-summary-14d)).

3. Data is manually retrieved in JSON format from the API. The `generate_function_from_openapi` function returns data in a JSON format and converts it to a pandas dataframe. Comparing the manually retrieved JSON data with the converted dataframe allows the user to verify the accuracy of the results.

4. Dataframes generated from the `generate_function_from_openapi` function and manually from the API are compared to each other to ensure that they match.

**Results**
*  The dataframes generated from the `generate_function_from_openapi` function and manually from the API have columns in a different order, but their outputs are identical.

*  In cases where the `generate_function_from_openapi` function could not return data, directly drawing data from the API also failed to return data. This confirms that there is genuenly no data avalible for the specific parameters for these endpoints.

## Section 1) Functions

In the section below a series of functions are provided. The first code block contains the function to convert the json output to pandas dataframe. The second code block is the output from the `generate_function_from_openapi` function. The third code block contains three functions to test the accuracy of the output. The three functions are;

1. fetch_json: Retrieves data in JSON format (this is a seperate function used for testing to ensure that mistakes from the original `json_to_dataframe_with_metadata` function do not carry over)
2. fetch_data: Retrieves data from the API as a dataframe
3. df_equal: Determins if the dataframe from the `generate_function_from_openapi` function and the fetch_data function are equal

In [257]:
def json_to_dataframe_with_metadata(data):
    # Extract metadata
    if isinstance(data, dict):
        metadata = data.get('metadata', [])
        if isinstance(metadata, dict):
            df_meta = pd.DataFrame.from_dict(metadata, orient='index').T
        else:
            df_meta = "No metadata provided"
    else:
        df_meta = "No metadata provided"

    # Normalize the JSON data into a flat table
    df = pd.json_normalize(data)

    # Identify columns with lists of dictionaries
    columns_with_list = [col for col in df.columns if isinstance(df[col].iloc[0], list)]

    # Explode columns containing lists
    for column in columns_with_list:
        df = df.explode(column)

        # Extract keys from dictionaries and use them as new column names
        dicts = df[column].dropna()

        if not dicts.empty:
            keys = set(k for d in dicts for k in (d.keys() if isinstance(d, dict) else []))
            for key in keys:
                df[key] = df[column].apply(lambda x: x.get(key) if isinstance(x, dict) else None)

        # Drop the original column containing dictionaries
        df.drop(columns=[column], inplace=True)

    return df, df_meta

### Replace the following code block with the output from the `generate_function_from_openapi`

In [258]:
def get_generation_availability_summary_14d(json):

    """
    Get the Fourteen-day generation forecast
    This function requests data from the '/generation/availability/summary/14D' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-generation-availability-summary-14D for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/generation/availability/summary/14D'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_generation_availability_summary_3yw(json):

    """
    Get the Three-year generation forecast
    This function requests data from the '/generation/availability/summary/3YW' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-generation-availability-summary-3YW for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/generation/availability/summary/3YW'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_availability_daily(level, bmUnit, fuelType, json):

    """
    Get the Fourteen-day generation capacity forecast (FOU2T14D)
    This function requests data from the '/forecast/availability/daily' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-availability-daily for more information

    Args:
        level:  Use date_total, fuelType, bmUnit.
        bmUnit:
        fuelType:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/availability/daily'

    params = {
        'level': level,
        'bmUnit': bmUnit,
        'fuelType': fuelType,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_availability_weekly(level, bmUnit, fuelType, json):

    """
    Get the Three-year generation capacity forecast (FOU2T3YW)
    This function requests data from the '/forecast/availability/weekly' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-availability-weekly for more information

    Args:
        level:  Use date_total, fuelType, bmUnit.
        bmUnit:
        fuelType:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/availability/weekly'

    params = {
        'level': level,
        'bmUnit': bmUnit,
        'fuelType': fuelType,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_availability_daily_history_publishtime_publishtime(publishTime, level, bmUnit, fuelType, json):

    """
    Get the History of the fourteen-day generation capacity forecast (FOU2T14D)
    This function requests data from the '/forecast/availability/daily/history' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-availability-daily-history for more information

    Args:
        publishTime: Format - date-time (as date-time in RFC3339).
        level:  Use date_total, fuelType, bmUnit.
        bmUnit:
        fuelType:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/availability/daily/history'

    params = {
        'publishTime': publishTime,
        'level': level,
        'bmUnit': bmUnit,
        'fuelType': fuelType,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_availability_weekly_history_publishtime_publishtime(publishTime, level, bmUnit, fuelType, json):

    """
    Get the History of the three-year generation capacity forecast (FOU2T3YW)
    This function requests data from the '/forecast/availability/weekly/history' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-availability-weekly-history for more information

    Args:
        publishTime: Format - date-time (as date-time in RFC3339).
        level:  Use date_total, fuelType, bmUnit.
        bmUnit:
        fuelType:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/availability/weekly/history'

    params = {
        'publishTime': publishTime,
        'level': level,
        'bmUnit': bmUnit,
        'fuelType': fuelType,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_availability_daily_evolution_forecastdate_forecastdate(forecastDate, level, bmUnit, fuelType, json):

    """
    Get the Evolution of the fourteen-day generation capacity forecast over time (FOU2T14D)
    This function requests data from the '/forecast/availability/daily/evolution' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-availability-daily-evolution for more information

    Args:
        forecastDate: Format - date (as full-date in RFC3339). The forecast date for the filter. This must be in the 'json' yyyy-MM-dd.
        level:  Use date_total, fuelType, bmUnit.
        bmUnit:
        fuelType:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/availability/daily/evolution'

    params = {
        'forecastDate': forecastDate,
        'level': level,
        'bmUnit': bmUnit,
        'fuelType': fuelType,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_availability_weekly_evolution_year_year_week_week(year, week, level, bmUnit, fuelType, json):

    """
    Get the Evolution of the three-year generation capacity forecast over time (FOU2T3YW)
    This function requests data from the '/forecast/availability/weekly/evolution' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-availability-weekly-evolution for more information

    Args:
        year: Format - int32.
        week: Format - int32.
        level:  Use date_total, fuelType, bmUnit.
        bmUnit:
        fuelType:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/availability/weekly/evolution'

    params = {
        'year': year,
        'week': week,
        'level': level,
        'bmUnit': bmUnit,
        'fuelType': fuelType,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_dynamic_bmunit_bmunit_snapshotat_snapshotat(bmUnit, snapshotAt, until, snapshotAtSettlementPeriod, untilSettlementPeriod, dataset, json):

    """
    Get the Dynamic data per BMU (SEL, SIL, MZT, MNZT, MDV, MDP, NTB, NTO, NDZ)
    This function requests data from the '/balancing/dynamic' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-dynamic for more information

    Args:
        bmUnit: The BM Unit date_to query.
        snapshotAt: Format - date-time (as date-time in RFC3339). When date_to retrieve a snapshot of data at.
That is, the latest datapoint before this time will be returned for each dataset.
        until: Format - date-time (as date-time in RFC3339). When date_to retrieve data until.
Data date_from the snapshot until this time will be returned.
        snapshotAtSettlementPeriod: Format - int32. The settlement period date_to retrieve a snapshot of data at.
If provided, the time part of SnapshotAt will be ignored.
        untilSettlementPeriod: Format - int32. The settlement period date_to retrieve data until.
If provided, the time part of Until will be ignored.
        dataset: Datasets date_to filter. If omitted, all datasets will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/dynamic'

    params = {
        'bmUnit': bmUnit,
        'snapshotAt': snapshotAt,
        'until': until,
        'snapshotAtSettlementPeriod': snapshotAtSettlementPeriod,
        'untilSettlementPeriod': untilSettlementPeriod,
        'dataset': dataset,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_fuelhh(publishDateTimeFrom, publishDateTimeTo, settlementDateFrom, settlementDateTo, settlementPeriod, fuelType, json):

    """
    Get the Half-hourly generation outturn by fuel type (FUELHH)
    This function requests data from the '/datasets/FUELHH' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-FUELHH for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        settlementDateFrom: Format - date (as full-date in RFC3339).
        settlementDateTo: Format - date (as full-date in RFC3339).
        settlementPeriod: List of Settlement Periods
        fuelType: Fuel Type e.g. NUCLEAR
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/FUELHH'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'settlementDateFrom': settlementDateFrom,
        'settlementDateTo': settlementDateTo,
        'settlementPeriod': settlementPeriod,
        'fuelType': fuelType,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_fuelinst(publishDateTimeFrom, publishDateTimeTo, settlementDateFrom, settlementDateTo, settlementPeriod, fuelType, json):

    """
    Get the Instantaneous generation outturn by fuel type (FUELINST)
    This function requests data from the '/datasets/FUELINST' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-FUELINST for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        settlementDateFrom: Format - date (as full-date in RFC3339).
        settlementDateTo: Format - date (as full-date in RFC3339).
        settlementPeriod: List of Settlement Periods
        fuelType: Fuel Type e.g. NUCLEAR
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/FUELINST'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'settlementDateFrom': settlementDateFrom,
        'settlementDateTo': settlementDateTo,
        'settlementPeriod': settlementPeriod,
        'fuelType': fuelType,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_uou2t14d(fuelType, publishDateTimeFrom, publishDateTimeTo, bmUnit, json):

    """
    Get the 2 to 14 days ahead generation availability aggregated by Balancing Mechanism Units (UOU2T14D)
    This function requests data from the '/datasets/UOU2T14D' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-UOU2T14D for more information

    Args:
        fuelType: The fuel type date_to query. Add each fuel type separately. If no fuel types are supplied, all fuel types will be returned.
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339). Start of the Publish Time range date_to query. If specified, PublishDateTimeTo must also be specified.
If both are omitted, latest published data is returned.
        publishDateTimeTo: Format - date-time (as date-time in RFC3339). End of the Publish Time range date_to query. If specified, PublishDateTimeFrom must also be specified.
If both are omitted, latest published data is returned.
        bmUnit: The BM units date_to query. Add each unit separately. Either the Elexon ID or the National Grid ID can be used.
If no BM unit is supplied all BM units will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/UOU2T14D'

    params = {
        'fuelType': fuelType,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_uou2t14d_stream(fuelType, publishDateTimeFrom, publishDateTimeTo, bmUnit):

    """
    Get the 2 to 14 days ahead generation availability aggregated by Balancing Mechanism Units (UOU2T14D) stream
    This function requests data from the '/datasets/UOU2T14D/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-UOU2T14D-stream for more information

    Args:
        fuelType: The fuel type date_to query. Add each fuel type separately. If no fuel types are supplied, all fuel types will be returned.
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339). Start of the Publish Time range date_to query. If specified, PublishDateTimeTo must also be specified.
If both are omitted, latest published data is returned.
        publishDateTimeTo: Format - date-time (as date-time in RFC3339). End of the Publish Time range date_to query. If specified, PublishDateTimeFrom must also be specified.
If both are omitted, latest published data is returned.
        bmUnit: The BM units date_to query. Add each unit separately. Either the Elexon ID or the National Grid ID can be used.
If no BM unit is supplied all BM units will be returned.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/UOU2T14D/stream'

    params = {
        'fuelType': fuelType,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_uou2t3yw(fuelType, publishDateTimeFrom, publishDateTimeTo, bmUnit, json):

    """
    Get the 2 to 156 weeks ahead generation availability aggregated by Balancing Mechanism Units (UOU2T3YW)
    This function requests data from the '/datasets/UOU2T3YW' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-UOU2T3YW for more information

    Args:
        fuelType: The fuel type date_to query. Add each fuel type separately. If no fuel types are supplied, all fuel types will be returned.
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339). Start of the Publish Time range date_to query. If specified, PublishDateTimeTo must also be specified.
If both are omitted, latest published data is returned.
        publishDateTimeTo: Format - date-time (as date-time in RFC3339). End of the Publish Time range date_to query. If specified, PublishDateTimeFrom must also be specified.
If both are omitted, latest published data is returned.
        bmUnit: The BM units date_to query. Add each unit separately. Either the Elexon ID or the National Grid ID can be used.
If no BM unit is supplied all BM units will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/UOU2T3YW'

    params = {
        'fuelType': fuelType,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_uou2t3yw_stream(fuelType, publishDateTimeFrom, publishDateTimeTo, bmUnit):

    """
    Get the 2 to 156 weeks ahead generation availability aggregated by Balancing Mechanism Units (UOU2T3YW) stream
    This function requests data from the '/datasets/UOU2T3YW/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-UOU2T3YW-stream for more information

    Args:
        fuelType: The fuel type date_to query. Add each fuel type separately. If no fuel types are supplied, all fuel types will be returned.
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339). Start of the Publish Time range date_to query. If specified, PublishDateTimeTo must also be specified.
If both are omitted, latest published data is returned.
        publishDateTimeTo: Format - date-time (as date-time in RFC3339). End of the Publish Time range date_to query. If specified, PublishDateTimeFrom must also be specified.
If both are omitted, latest published data is returned.
        bmUnit: The BM units date_to query. Add each unit separately. Either the Elexon ID or the National Grid ID can be used.
If no BM unit is supplied all BM units will be returned.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/UOU2T3YW/stream'

    params = {
        'fuelType': fuelType,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_fou2t14d(fuelType, publishDate, publishDateTimeFrom, publishDateTimeTo, biddingZone, interconnecdate_tor, json):

    """
    Get the 2 to 14 days ahead generation availability aggregated by fuel type (FOU2T14D)
    This function requests data from the '/datasets/FOU2T14D' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-FOU2T14D for more information

    Args:
        fuelType:
        publishDate: Format - date (as full-date in RFC3339). The publish date for filtering. This must be in the 'json' yyyy-MM-dd.
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        biddingZone:
        interconnecdate_tor:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/FOU2T14D'

    params = {
        'fuelType': fuelType,
        'publishDate': publishDate,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'biddingZone': biddingZone,
        'interconnector': interconnecdate_tor,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_fou2t3yw(fuelType, publishDate, publishDateTimeFrom, publishDateTimeTo, week, year, biddingZone, interconnecdate_tor, json):

    """
    Get the 2 to 156 weeks ahead generation availability aggregated by fuel type (FOU2T3YW)
    This function requests data from the '/datasets/FOU2T3YW' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-FOU2T3YW for more information

    Args:
        fuelType:
        publishDate: Format - date (as full-date in RFC3339). The publish date for filtering. This must be in the 'json' yyyy-MM-dd.
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        week:
        year:
        biddingZone:
        interconnecdate_tor:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/FOU2T3YW'

    params = {
        'fuelType': fuelType,
        'publishDate': publishDate,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'week': week,
        'year': year,
        'biddingZone': biddingZone,
        'interconnector': interconnecdate_tor,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_nou2t14d(publishDate, publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the 2 to 14 days ahead generation availability aggregated data (NOU2T14D)
    This function requests data from the '/datasets/NOU2T14D' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NOU2T14D for more information

    Args:
        publishDate: Format - date (as full-date in RFC3339). The publish date for filtering. This must be in the 'json' yyyy-MM-dd.
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NOU2T14D'

    params = {
        'publishDate': publishDate,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_nou2t3yw(publishDate, publishDateTimeFrom, publishDateTimeTo, week, year, json):

    """
    Get the 2 to 156 weeks ahead generation availability aggregated data (NOU2T3YW)
    This function requests data from the '/datasets/NOU2T3YW' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NOU2T3YW for more information

    Args:
        publishDate: Format - date (as full-date in RFC3339). The publish date for filtering. This must be in the 'json' yyyy-MM-dd.
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        week:
        year:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NOU2T3YW'

    params = {
        'publishDate': publishDate,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'week': week,
        'year': year,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_temp(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Temperature data (TEMP)
    This function requests data from the '/datasets/TEMP' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-TEMP for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/TEMP'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_indgen(boundary, publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Day and day-ahead indicated generation (INDGEN)
    This function requests data from the '/datasets/INDGEN' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-INDGEN for more information

    Args:
        boundary:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/INDGEN'

    params = {
        'boundary': boundary,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_indgen_stream(boundary, publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Day and day-ahead indicated generation (INDGEN) stream
    This function requests data from the '/datasets/INDGEN/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-INDGEN-stream for more information

    Args:
        boundary:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/INDGEN/stream'

    params = {
        'boundary': boundary,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_inddem(boundary, publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Day and day-ahead indicated demand (INDDEM)
    This function requests data from the '/datasets/INDDEM' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-INDDEM for more information

    Args:
        boundary:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/INDDEM'

    params = {
        'boundary': boundary,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_inddem_stream(boundary, publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Day and day-ahead indicated demand (INDDEM) stream
    This function requests data from the '/datasets/INDDEM/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-INDDEM-stream for more information

    Args:
        boundary:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/INDDEM/stream'

    params = {
        'boundary': boundary,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_melngc(boundary, publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Day and day-ahead indicated margin (MELNGC)
    This function requests data from the '/datasets/MELNGC' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MELNGC for more information

    Args:
        boundary:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MELNGC'

    params = {
        'boundary': boundary,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_melngc_stream(boundary, publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Day and day-ahead indicated margin (MELNGC) stream
    This function requests data from the '/datasets/MELNGC/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MELNGC-stream for more information

    Args:
        boundary:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MELNGC/stream'

    params = {
        'boundary': boundary,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_imbalngc(boundary, publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Day and day-ahead indicated imbalance (IMBALNGC)
    This function requests data from the '/datasets/IMBALNGC' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-IMBALNGC for more information

    Args:
        boundary:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/IMBALNGC'

    params = {
        'boundary': boundary,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_imbalngc_stream(boundary, publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Day and day-ahead indicated imbalance (IMBALNGC) stream
    This function requests data from the '/datasets/IMBALNGC/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-IMBALNGC-stream for more information

    Args:
        boundary:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/IMBALNGC/stream'

    params = {
        'boundary': boundary,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ndf(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Day and day-ahead National Demand forecast (NDF)
    This function requests data from the '/datasets/NDF' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NDF for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NDF'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ndf_stream(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Day and day-ahead National Demand forecast (NDF) stream
    This function requests data from the '/datasets/NDF/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NDF-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NDF/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_tsdf(boundary, publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Day and day-ahead Transmission System Demand forecast (TSDF)
    This function requests data from the '/datasets/TSDF' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-TSDF for more information

    Args:
        boundary:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/TSDF'

    params = {
        'boundary': boundary,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_tsdf_stream(boundary, publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Day and day-ahead Transmission System Demand forecast (TSDF) stream
    This function requests data from the '/datasets/TSDF/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-TSDF-stream for more information

    Args:
        boundary:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/TSDF/stream'

    params = {
        'boundary': boundary,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_windfor(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Wind generation forecast (WINDFOR)
    This function requests data from the '/datasets/WINDFOR' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-WINDFOR for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/WINDFOR'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_windfor_stream(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Wind generation forecast (WINDFOR) stream
    This function requests data from the '/datasets/WINDFOR/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-WINDFOR-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/WINDFOR/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_indo(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Initial National Demand outturn (INDO)
    This function requests data from the '/datasets/INDO' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-INDO for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/INDO'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_itsdo(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Initial Transmission System Demand outturn (ITSDO)
    This function requests data from the '/datasets/ITSDO' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-ITSDO for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/ITSDO'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ndfd(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the 2-14 days ahead National Demand and surplus forecast (NDFD)
    This function requests data from the '/datasets/NDFD' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NDFD for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NDFD'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ndfd_stream(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the 2-14 days ahead National Demand and surplus forecast (NDFD) stream
    This function requests data from the '/datasets/NDFD/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NDFD-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NDFD/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_tsdfd(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the 2-14 days ahead Transmission System Demand and surplus forecast (TSDFD)
    This function requests data from the '/datasets/TSDFD' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-TSDFD for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/TSDFD'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_tsdfd_stream(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the 2-14 days ahead Transmission System Demand and surplus forecast (TSDFD) stream
    This function requests data from the '/datasets/TSDFD/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-TSDFD-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/TSDFD/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ndfw(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the 2-52 weeks ahead National Demand and surplus forecast (NDFW)
    This function requests data from the '/datasets/NDFW' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NDFW for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NDFW'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ndfw_stream(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the 2-52 weeks ahead National Demand and surplus forecast (NDFW) stream
    This function requests data from the '/datasets/NDFW/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NDFW-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NDFW/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_tsdfw(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the 2-52 weeks ahead Transmission System Demand and surplus forecast (TSDFW)
    This function requests data from the '/datasets/TSDFW' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-TSDFW for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/TSDFW'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_tsdfw_stream(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the 2-52 weeks ahead Transmission System Demand and surplus forecast (TSDFW) stream
    This function requests data from the '/datasets/TSDFW/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-TSDFW-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/TSDFW/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_freq(measurementDateTimeFrom, measurementDateTimeTo, json):

    """
    Get the System frequency (FREQ)
    This function requests data from the '/datasets/FREQ' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-FREQ for more information

    Args:
        measurementDateTimeFrom: Format - date-time (as date-time in RFC3339).
        measurementDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/FREQ'

    params = {
        'measurementDateTimeFrom': measurementDateTimeFrom,
        'measurementDateTimeTo': measurementDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_freq_stream(measurementDateTimeFrom, measurementDateTimeTo):

    """
    Get the System frequency (FREQ) stream
    This function requests data from the '/datasets/FREQ/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-FREQ-stream for more information

    Args:
        measurementDateTimeFrom: Format - date-time (as date-time in RFC3339).
        measurementDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/FREQ/stream'

    params = {
        'measurementDateTimeFrom': measurementDateTimeFrom,
        'measurementDateTimeTo': measurementDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ocnmfd(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the 2-14 days ahead demand surplus forecast (OCNMFD)
    This function requests data from the '/datasets/OCNMFD' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-OCNMFD for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/OCNMFD'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ocnmfd_stream(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the 2-14 days ahead demand surplus forecast (OCNMFD) stream
    This function requests data from the '/datasets/OCNMFD/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-OCNMFD-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/OCNMFD/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ocnmfd2(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the 2-14 days ahead demand margin forecast (OCNMFD2)
    This function requests data from the '/datasets/OCNMFD2' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-OCNMFD2 for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/OCNMFD2'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ocnmfd2_stream(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the 2-14 days ahead demand margin forecast (OCNMFD2) stream
    This function requests data from the '/datasets/OCNMFD2/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-OCNMFD2-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/OCNMFD2/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ocnmf3y(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the 2-156 weeks ahead demand surplus forecast (OCNMF3Y)
    This function requests data from the '/datasets/OCNMF3Y' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-OCNMF3Y for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/OCNMF3Y'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ocnmf3y_stream(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the 2-156 weeks ahead demand surplus forecast (OCNMF3Y) stream
    This function requests data from the '/datasets/OCNMF3Y/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-OCNMF3Y-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/OCNMF3Y/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ocnmf3y2(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the 2-156 weeks ahead demand margin forecast (OCNMF3Y2)
    This function requests data from the '/datasets/OCNMF3Y2' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-OCNMF3Y2 for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/OCNMF3Y2'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ocnmf3y2_stream(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the 2-156 weeks ahead demand margin forecast (OCNMF3Y2) stream
    This function requests data from the '/datasets/OCNMF3Y2/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-OCNMF3Y2-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/OCNMF3Y2/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_metadata_latest(json):

    """
    Get the Returns the time when each dataset was last updated
    This function requests data from the '/datasets/metadata/latest' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-metadata-latest for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/metadata/latest'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_nonbm(date_from, date_to, json):

    """
    Get the Non-BM STOR (NONBM)
    This function requests data from the '/datasets/NONBM' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NONBM for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The start of the data publish time window.
        date_to: Format - date-time (as date-time in RFC3339). The end of the data publish time window.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NONBM'

    params = {
        'from': date_from,
        'to': date_to,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_nonbm_stream(date_from, date_to):

    """
    Get the Non-BM STOR (NONBM) stream
    This function requests data from the '/datasets/NONBM/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NONBM-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The start of the data publish time window.
        date_to: Format - date-time (as date-time in RFC3339). The end of the data publish time window.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NONBM/stream'

    params = {
        'from': date_from,
        'to': date_to,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_sil(date_from, date_to, bmUnit, json):

    """
    Get the Stable Import Limit (SIL)
    This function requests data from the '/datasets/SIL' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-SIL for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The start of the data time window.
        date_to: Format - date-time (as date-time in RFC3339). The end of the data time window.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/SIL'

    params = {
        'from': date_from,
        'to': date_to,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_sil_stream(date_from, date_to, bmUnit):

    """
    Get the Stable Import Limit (SIL) stream
    This function requests data from the '/datasets/SIL/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-SIL-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The start of the data time window.
        date_to: Format - date-time (as date-time in RFC3339). The end of the data time window.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/SIL/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_sel(date_from, date_to, bmUnit, json):

    """
    Get the Stable Export Limit (SEL)
    This function requests data from the '/datasets/SEL' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-SEL for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The start of the data time window.
        date_to: Format - date-time (as date-time in RFC3339). The end of the data time window.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/SEL'

    params = {
        'from': date_from,
        'to': date_to,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_sel_stream(date_from, date_to, bmUnit):

    """
    Get the Stable Export Limit (SEL) stream
    This function requests data from the '/datasets/SEL/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-SEL-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The start of the data time window.
        date_to: Format - date-time (as date-time in RFC3339). The end of the data time window.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/SEL/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_mzt_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Minimum Zero Time (MZT)
    This function requests data from the '/datasets/MZT' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MZT for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MZT'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_mzt_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Minimum Zero Time (MZT) stream
    This function requests data from the '/datasets/MZT/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MZT-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MZT/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_mnzt_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Minimum Non-Zero Time (MNZT)
    This function requests data from the '/datasets/MNZT' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MNZT for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MNZT'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_mnzt_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Minimum Non-Zero Time (MNZT) stream
    This function requests data from the '/datasets/MNZT/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MNZT-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MNZT/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_mdv_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Maximum Delivery Volume (MDV)
    This function requests data from the '/datasets/MDV' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MDV for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MDV'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_mdv_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Maximum Delivery Volume (MDV) stream
    This function requests data from the '/datasets/MDV/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MDV-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MDV/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_mdp_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Maximum Delivery Period (MDP)
    This function requests data from the '/datasets/MDP' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MDP for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MDP'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_mdp_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Maximum Delivery Period (MDP) stream
    This function requests data from the '/datasets/MDP/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MDP-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MDP/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ntb_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Notice to Deliver Bids (NTB)
    This function requests data from the '/datasets/NTB' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NTB for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NTB'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ntb_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Notice to Deliver Bids (NTB) stream
    This function requests data from the '/datasets/NTB/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NTB-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NTB/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_nto_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Notice to Deliver Offers (NTO)
    This function requests data from the '/datasets/NTO' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NTO for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NTO'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_nto_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Notice to Deliver Offers (NTO) stream
    This function requests data from the '/datasets/NTO/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NTO-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NTO/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ndz_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Notice to Deviate from Zero (NDZ)
    This function requests data from the '/datasets/NDZ' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NDZ for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NDZ'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ndz_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Notice to Deviate from Zero (NDZ) stream
    This function requests data from the '/datasets/NDZ/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NDZ-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NDZ/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_qas_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Balancing Services Volume (QAS)
    This function requests data from the '/datasets/QAS' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-QAS for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/QAS'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_qas_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Balancing Services Volume (QAS) stream
    This function requests data from the '/datasets/QAS/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-QAS-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/QAS/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand(json):

    """
    Get the This endpoint is obsolete, and this location may be removed with no further notice.
    This function requests data from the '/demand' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_stream(json):

    """
    Get the This endpoint is obsolete, and this location may be removed with no further notice.
    This function requests data from the '/demand/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-stream for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/stream'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_rollingsystemdemand(json):

    """
    Get the This endpoint is obsolete, and this location may be removed with no further notice.
    This function requests data from the '/demand/rollingSystemDemand' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-rollingSystemDemand for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/rollingSystemDemand'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_summary(json):

    """
    Get the This endpoint is obsolete, and this location may be removed with no further notice.
    This function requests data from the '/demand/summary' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-summary for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/summary'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_peak(date_from, date_to, json):

    """
    Get the Peak demand per day (ITSDO)
    This function requests data from the '/demand/peak' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-peak for more information

    Args:
        date_from: Format - date (as full-date in RFC3339). The start of the requested date range.
        date_to: Format - date (as full-date in RFC3339). The end of the requested date range.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/peak'

    params = {
        'from': date_from,
        'to': date_to,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_day_ahead(boundary, json):

    """
    Get the Day-ahead demand forecast (NDF, TSDF)
    This function requests data from the '/forecast/demand/day-ahead' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-day-ahead for more information

    Args:
        boundary: Omitting this will return only national data. Specifying boundary=zonal will return only zonal data.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/day-ahead'

    params = {
        'boundary': boundary,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_daily(json):

    """
    Get the Fourteen day demand forecast (NDFD, TSDFD)
    This function requests data from the '/forecast/demand/daily' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-daily for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/daily'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_weekly(json):

    """
    Get the One-year demand forecast (NDFW, TSDFW)
    This function requests data from the '/forecast/demand/weekly' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-weekly for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/weekly'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_day_ahead_history_publishtime_publishtime(publishTime, boundary, json):

    """
    Get the History of the day-ahead demand forecast (NDF, TSDF)
    This function requests data from the '/forecast/demand/day-ahead/history' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-day-ahead-history for more information

    Args:
        publishTime: Format - date-time (as date-time in RFC3339).
        boundary: Omitting this will return only national data. Specifying boundary=zonal will return only zonal data.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/day-ahead/history'

    params = {
        'publishTime': publishTime,
        'boundary': boundary,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_daily_history_publishtime_publishtime(publishTime, json):

    """
    Get the History of the fourteen-day demand forecast (NDFD, TSDFD)
    This function requests data from the '/forecast/demand/daily/history' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-daily-history for more information

    Args:
        publishTime: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/daily/history'

    params = {
        'publishTime': publishTime,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_weekly_history_publishtime_publishtime(publishTime, json):

    """
    Get the History of the one-year demand forecast (NDFW, TSDFW)
    This function requests data from the '/forecast/demand/weekly/history' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-weekly-history for more information

    Args:
        publishTime: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/weekly/history'

    params = {
        'publishTime': publishTime,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_day_ahead_evolution_settlementdate_settlementdate_settle(settlementDate, settlementPeriod, boundary, json):

    """
    Get the Evolution of the day-ahead demand forecast over time (NDF, TSDF)
    This function requests data from the '/forecast/demand/day-ahead/evolution' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-day-ahead-evolution for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). The settlement date for the filter. This must be in the 'json' yyyy-MM-dd.
        settlementPeriod:
        boundary: Omitting this will return only national data. Specifying boundary=zonal will return only zonal data.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/day-ahead/evolution'

    params = {
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'boundary': boundary,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_daily_evolution_forecastdate_forecastdate(forecastDate, json):

    """
    Get the Evolution of the fourteen-day demand forecast over time (NDFD, TSDFD)
    This function requests data from the '/forecast/demand/daily/evolution' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-daily-evolution for more information

    Args:
        forecastDate: Format - date (as full-date in RFC3339). The forecast date for the filter. This must be in the 'json' yyyy-MM-dd.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/daily/evolution'

    params = {
        'forecastDate': forecastDate,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_weekly_evolution_forecastyear_forecastyear_forecastweek(forecastYear, forecastWeek, json):

    """
    Get the Evolution of the one-year demand forecast over time  (NDFW, TSDFW)
    This function requests data from the '/forecast/demand/weekly/evolution' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-weekly-evolution for more information

    Args:
        forecastYear: Format - int32.
        forecastWeek: Format - int32.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/weekly/evolution'

    params = {
        'forecastYear': forecastYear,
        'forecastWeek': forecastWeek,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_day_ahead_latest_from_from_to_to(date_from, date_to, boundary, json):

    """
    Get the Historic view of the latest forecasted demand (NDF, TSDF)
    This function requests data from the '/forecast/demand/day-ahead/latest' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-day-ahead-latest for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        boundary: Omitting this will return only national data. Specifying boundary=zonal will return only zonal data.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/day-ahead/latest'

    params = {
        'from': date_from,
        'to': date_to,
        'boundary': boundary,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_day_ahead_latest_stream_from_from_to_to(date_from, date_to, boundary):

    """
    Get the Historic view of the latest forecasted demand (NDF, TSDF) stream
    This function requests data from the '/forecast/demand/day-ahead/latest/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-day-ahead-latest-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        boundary: Omitting this will return only national data. Specifying boundary=zonal will return only zonal data.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/day-ahead/latest/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'boundary': boundary,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_day_ahead_earliest_from_from_to_to(date_from, date_to, boundary, json):

    """
    Get the Historic view of the earliest forecasted demand (NDF, TSDF)
    This function requests data from the '/forecast/demand/day-ahead/earliest' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-day-ahead-earliest for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        boundary: Omitting this will return only national data. Specifying boundary=zonal will return only zonal data.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/day-ahead/earliest'

    params = {
        'from': date_from,
        'to': date_to,
        'boundary': boundary,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_day_ahead_earliest_stream_from_from_to_to(date_from, date_to, boundary):

    """
    Get the Historic view of the earliest forecasted demand (NDF, TSDF) stream
    This function requests data from the '/forecast/demand/day-ahead/earliest/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-day-ahead-earliest-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        boundary: Omitting this will return only national data. Specifying boundary=zonal will return only zonal data.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/day-ahead/earliest/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'boundary': boundary,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_day_ahead_peak(boundary, date_from, date_to, json):

    """
    Get the Peak forecasted demand per day (TSDF)
    This function requests data from the '/forecast/demand/day-ahead/peak' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-day-ahead-peak for more information

    Args:
        boundary:
        date_from: Format - date (as full-date in RFC3339). The start of the requested date range.
        date_to: Format - date (as full-date in RFC3339). The end of the requested date range.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/day-ahead/peak'

    params = {
        'boundary': boundary,
        'from': date_from,
        'to': date_to,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_margin_daily(json):

    """
    Get the Daily margin forecast (OCNMFD2)
    This function requests data from the '/forecast/margin/daily' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-margin-daily for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/margin/daily'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_margin_daily_history_publishtime_publishtime(publishTime, json):

    """
    Get the Historical daily margin forecast (OCNMFD2)
    This function requests data from the '/forecast/margin/daily/history' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-margin-daily-history for more information

    Args:
        publishTime: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/margin/daily/history'

    params = {
        'publishTime': publishTime,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_margin_daily_evolution_forecastdate_forecastdate(forecastDate, json):

    """
    Get the Evolution daily margin forecast (OCNMFD2)
    This function requests data from the '/forecast/margin/daily/evolution' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-margin-daily-evolution for more information

    Args:
        forecastDate: Format - date (as full-date in RFC3339). The forecast date for the filter. This must be in the 'json' yyyy-MM-dd.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/margin/daily/evolution'

    params = {
        'forecastDate': forecastDate,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_margin_weekly(range, json):

    """
    Get the Weekly margin forecast (OCNMFW2, OCNMF3Y2)
    This function requests data from the '/forecast/margin/weekly' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-margin-weekly for more information

    Args:
        range:  Use 52W, 156W.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/margin/weekly'

    params = {
        'range': range,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_margin_weekly_history_publishtime_publishtime(publishTime, range, json):

    """
    Get the Historical weekly margin forecast (OCNMFW2, OCNMF3Y2)
    This function requests data from the '/forecast/margin/weekly/history' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-margin-weekly-history for more information

    Args:
        publishTime: Format - date-time (as date-time in RFC3339).
        range:  Use 52W, 156W.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/margin/weekly/history'

    params = {
        'publishTime': publishTime,
        'range': range,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_margin_weekly_evolution_year_year_week_week(year, week, range, json):

    """
    Get the Evolution daily margin forecast (OCNMFW2, OCNMF3Y2)
    This function requests data from the '/forecast/margin/weekly/evolution' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-margin-weekly-evolution for more information

    Args:
        year: Format - int32.
        week: Format - int32.
        range:  Use 52W, 156W.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/margin/weekly/evolution'

    params = {
        'year': year,
        'week': week,
        'range': range,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_surplus_daily(json):

    """
    Get the Daily surplus forecast (OCNMFD)
    This function requests data from the '/forecast/surplus/daily' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-surplus-daily for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/surplus/daily'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_surplus_daily_history_publishtime_publishtime(publishTime, json):

    """
    Get the Historical daily surplus forecast (OCNMFD)
    This function requests data from the '/forecast/surplus/daily/history' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-surplus-daily-history for more information

    Args:
        publishTime: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/surplus/daily/history'

    params = {
        'publishTime': publishTime,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_surplus_daily_evolution_forecastdate_forecastdate(forecastDate, json):

    """
    Get the Evolution daily surplus forecast (OCNMFD)
    This function requests data from the '/forecast/surplus/daily/evolution' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-surplus-daily-evolution for more information

    Args:
        forecastDate: Format - date (as full-date in RFC3339). The forecast date for the filter. This must be in the 'json' yyyy-MM-dd.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/surplus/daily/evolution'

    params = {
        'forecastDate': forecastDate,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_surplus_weekly(range, json):

    """
    Get the Weekly surplus forecast (OCNMFW, OCNMF3Y)
    This function requests data from the '/forecast/surplus/weekly' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-surplus-weekly for more information

    Args:
        range:  Use 52W, 156W.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/surplus/weekly'

    params = {
        'range': range,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_surplus_weekly_history_publishtime_publishtime(publishTime, range, json):

    """
    Get the Historical weekly surplus forecast (OCNMFW, OCNMF3Y)
    This function requests data from the '/forecast/surplus/weekly/history' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-surplus-weekly-history for more information

    Args:
        publishTime: Format - date-time (as date-time in RFC3339).
        range:  Use 52W, 156W.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/surplus/weekly/history'

    params = {
        'publishTime': publishTime,
        'range': range,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_surplus_weekly_evolution_year_year_week_week(year, week, range, json):

    """
    Get the Evolution weekly surplus forecast (OCNMFW, OCNMF3Y)
    This function requests data from the '/forecast/surplus/weekly/evolution' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-surplus-weekly-evolution for more information

    Args:
        year: Format - int32.
        week: Format - int32.
        range:  Use 52W, 156W.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/surplus/weekly/evolution'

    params = {
        'year': year,
        'week': week,
        'range': range,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_health():

    """
    Get the Health check
    This function requests data from the '/health' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-health for more information

    Args:

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/health'

    params = {
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_indicated_day_ahead(boundary, json):

    """
    Get the Latest indicated day-ahead forecast (INDDEM, INDGEN, IMBALNGC, MELNGC)
    This function requests data from the '/forecast/indicated/day-ahead' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-indicated-day-ahead for more information

    Args:
        boundary: Omitting this will return only national data. Specifying boundary=zonal will return only zonal data.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/indicated/day-ahead'

    params = {
        'boundary': boundary,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_indicated_day_ahead_history_publishtime_publishtime(publishTime, boundary, json):

    """
    Get the Historical indicated day-ahead forecast (INDDEM, INDGEN, IMBALNGC, MELNGC)
    This function requests data from the '/forecast/indicated/day-ahead/history' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-indicated-day-ahead-history for more information

    Args:
        publishTime: Format - date-time (as date-time in RFC3339).
        boundary: Omitting this will return only national data. Specifying boundary=zonal will return only zonal data.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/indicated/day-ahead/history'

    params = {
        'publishTime': publishTime,
        'boundary': boundary,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_indicated_day_ahead_evolution_settlementdate_settlementdate_set(settlementDate, settlementPeriod, boundary, json):

    """
    Get the Evolution indicated day-ahead forecast (INDDEM, INDGEN, IMBALNGC, MELNGC)
    This function requests data from the '/forecast/indicated/day-ahead/evolution' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-indicated-day-ahead-evolution for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). The settlement date for the filter. This must be in the 'json' yyyy-MM-dd.
        settlementPeriod:
        boundary: Omitting this will return only national data. Specifying boundary=zonal will return only zonal data.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/indicated/day-ahead/evolution'

    params = {
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'boundary': boundary,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_nonbm_stor_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, includeZero, json):

    """
    Get the Non-BM STOR time series (NONBM)
    This function requests data from the '/balancing/nonbm/stor' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-nonbm-stor for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        includeZero: Include data points with a generation of zero.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/nonbm/stor'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'includeZero': includeZero,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_nonbm_stor_events_count_count(count, before, settlementPeriodBefore, json):

    """
    Get the Non-BM STOR events (NONBM)
    This function requests data from the '/balancing/nonbm/stor/events' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-nonbm-stor-events for more information

    Args:
        count: Format - int32. The number of events date_to return.
        before: Format - date-time (as date-time in RFC3339). If specified, filters events date_to those with a start time before or at the date, or a settlement date before the date if
settlementPeriodBefore is also specified.
If omitted, latest events are returned.
        settlementPeriodBefore: Format - int32. Filters events date_to those with a settlement period before or at the value.
Before parameter must be specified if this is specified.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/nonbm/stor/events'

    params = {
        'count': count,
        'before': before,
        'settlementPeriodBefore': settlementPeriodBefore,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_generation_outturn_halfhourlyinterconnector(json):

    """
    Get the This endpoint is obsolete, and this location may be removed with no further notice.
    This function requests data from the '/generation/outturn/halfHourlyInterconnector' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-generation-outturn-halfHourlyInterconnector for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/generation/outturn/halfHourlyInterconnector'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_generation_outturn_fuelinsthhcur(json):

    """
    Get the This endpoint is obsolete, and this location may be removed with no further notice.
    This function requests data from the '/generation/outturn/FUELINSTHHCUR' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-generation-outturn-FUELINSTHHCUR for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/generation/outturn/FUELINSTHHCUR'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_generation_outturn_summary(startTime, endTime, includeNegativeGeneration, json):

    """
    Get the Historic generation automatically down-sampled (FUELINST)
    This function requests data from the '/generation/outturn/summary' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-generation-outturn-summary for more information

    Args:
        startTime: Format - date-time (as date-time in RFC3339).
        endTime: Format - date-time (as date-time in RFC3339).
        includeNegativeGeneration:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/generation/outturn/summary'

    params = {
        'startTime': startTime,
        'endTime': endTime,
        'includeNegativeGeneration': includeNegativeGeneration,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_reference_fueltypes_all():

    """
    Get the Fuel types
    This function requests data from the '/reference/fueltypes/all' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-reference-fueltypes-all for more information

    Args:

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/reference/fueltypes/all'

    params = {
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_reference_bmunits_all():

    """
    Get the BM Units
    This function requests data from the '/reference/bmunits/all' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-reference-bmunits-all for more information

    Args:

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/reference/bmunits/all'

    params = {
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_reference_interconnectors_all():

    """
    Get the Interconnectors
    This function requests data from the '/reference/interconnectors/all' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-reference-interconnectors-all for more information

    Args:

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/reference/interconnectors/all'

    params = {
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_system_frequency(date_from, date_to, json):

    """
    Get the System frequency (FREQ)
    This function requests data from the '/system/frequency' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-system-frequency for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/system/frequency'

    params = {
        'from': date_from,
        'to': date_to,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_system_frequency_stream(date_from, date_to):

    """
    Get the System frequency (FREQ) stream
    This function requests data from the '/system/frequency/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-system-frequency-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/system/frequency/stream'

    params = {
        'from': date_from,
        'to': date_to,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_temperature(date_from, date_to, json):

    """
    Get the Temperature data (TEMP)
    This function requests data from the '/temperature' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-temperature for more information

    Args:
        date_from: Format - date (as full-date in RFC3339). The date_from date for the filter. This must be in the 'json' yyyy-MM-dd.
        date_to: Format - date (as full-date in RFC3339). The date_to date for the filter. This must be in the 'json' yyyy-MM-dd.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/temperature'

    params = {
        'from': date_from,
        'to': date_to,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_generation_wind(json):

    """
    Get the Current wind generation forecast (WINDFOR)
    This function requests data from the '/forecast/generation/wind' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-generation-wind for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/generation/wind'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_generation_wind_history_publishtime_publishtime(publishTime, json):

    """
    Get the History of the wind generation forecast (WINDFOR)
    This function requests data from the '/forecast/generation/wind/history' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-generation-wind-history for more information

    Args:
        publishTime: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/generation/wind/history'

    params = {
        'publishTime': publishTime,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_generation_wind_evolution_starttime_starttime(startTime, json):

    """
    Get the Evolution of the wind generation forecast over time (WINDFOR)
    This function requests data from the '/forecast/generation/wind/evolution' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-generation-wind-evolution for more information

    Args:
        startTime: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/generation/wind/evolution'

    params = {
        'startTime': startTime,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_generation_wind_latest_from_from_to_to(date_from, date_to, json):

    """
    Get the Historic view of the latest forecasted wind generation (WINDFOR)
    This function requests data from the '/forecast/generation/wind/latest' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-generation-wind-latest for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/generation/wind/latest'

    params = {
        'from': date_from,
        'to': date_to,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_generation_wind_latest_stream_from_from_to_to(date_from, date_to):

    """
    Get the Historic view of the latest forecasted wind generation (WINDFOR) stream
    This function requests data from the '/forecast/generation/wind/latest/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-generation-wind-latest-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/generation/wind/latest/stream'

    params = {
        'from': date_from,
        'to': date_to,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_generation_wind_earliest_from_from_to_to(date_from, date_to, json):

    """
    Get the Historic view of the earliest forecasted wind generation (WINDFOR)
    This function requests data from the '/forecast/generation/wind/earliest' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-generation-wind-earliest for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/generation/wind/earliest'

    params = {
        'from': date_from,
        'to': date_to,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_generation_wind_earliest_stream_from_from_to_to(date_from, date_to):

    """
    Get the Historic view of the earliest forecasted wind generation (WINDFOR) stream
    This function requests data from the '/forecast/generation/wind/earliest/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-generation-wind-earliest-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/generation/wind/earliest/stream'

    params = {
        'from': date_from,
        'to': date_to,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_generation_wind_peak(date_from, date_to, json):

    """
    Get the Peak wind generation forecast for each day (WINDFOR)
    This function requests data from the '/forecast/generation/wind/peak' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-generation-wind-peak for more information

    Args:
        date_from: Format - date (as full-date in RFC3339). The start of the requested date range.
        date_to: Format - date (as full-date in RFC3339). The end of the requested date range.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/generation/wind/peak'

    params = {
        'from': date_from,
        'to': date_to,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_dynamic_rates_bmunit_bmunit_snapshotat_snapshotat(bmUnit, snapshotAt, until, snapshotAtSettlementPeriod, untilSettlementPeriod, dataset, json):

    """
    Get the Rate data per BMU (RDRE, RURE, RDRI, RURI)
    This function requests data from the '/balancing/dynamic/rates' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-dynamic-rates for more information

    Args:
        bmUnit: The BM Unit date_to query.
        snapshotAt: Format - date-time (as date-time in RFC3339). When date_to retrieve a snapshot of data at.
That is, the latest datapoint before this time will be returned for each dataset.
        until: Format - date-time (as date-time in RFC3339). When date_to retrieve data until.
Data date_from the snapshot until this time will be returned.
        snapshotAtSettlementPeriod: Format - int32. The settlement period date_to retrieve a snapshot of data at.
If provided, the time part of SnapshotAt will be ignored.
        untilSettlementPeriod: Format - int32. The settlement period date_to retrieve data until.
If provided, the time part of Until will be ignored.
        dataset: Datasets date_to filter. If empty, all datasets will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/dynamic/rates'

    params = {
        'bmUnit': bmUnit,
        'snapshotAt': snapshotAt,
        'until': until,
        'snapshotAtSettlementPeriod': snapshotAtSettlementPeriod,
        'untilSettlementPeriod': untilSettlementPeriod,
        'dataset': dataset,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_physical_bmunit_bmunit_from_from_to_to(bmUnit, date_from, date_to, settlementPeriodFrom, settlementPeriodTo, dataset, json):

    """
    Get the Physical data per BMU (PN, QPN, MILS, MELS)
    This function requests data from the '/balancing/physical' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-physical for more information

    Args:
        bmUnit: The BM Unit date_to query.
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        dataset: Datasets date_to filter. If empty, all datasets will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/physical'

    params = {
        'bmUnit': bmUnit,
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'dataset': dataset,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_nonbm_disbsad_summary_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, json):

    """
    Get the Disaggregated balancing services adjustment time series (DISBSAD)
    This function requests data from the '/balancing/nonbm/disbsad/summary' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-nonbm-disbsad-summary for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/nonbm/disbsad/summary'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_nonbm_volumes_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Balancing services volume (QAS)
    This function requests data from the '/balancing/nonbm/volumes' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-nonbm-volumes for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/nonbm/volumes'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_mid_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, dataProviders, json):

    """
    Get the Market Index Data (MID)
    This function requests data from the '/datasets/MID' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MID for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        dataProviders: The data providers date_to query. If no data provider is selected both will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MID'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'dataProviders': dataProviders,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_mid_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, dataProviders):

    """
    Get the Market Index Data (MID) stream
    This function requests data from the '/datasets/MID/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MID-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        dataProviders: The data providers date_to query. If no data provider is selected both will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MID/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'dataProviders': dataProviders,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_rure_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Run Up Rate Export (RURE)
    This function requests data from the '/datasets/RURE' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-RURE for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/RURE'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_rure_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Run Up Rate Export (RURE) stream
    This function requests data from the '/datasets/RURE/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-RURE-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/RURE/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_rdre_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Run Down Rate Export (RDRE)
    This function requests data from the '/datasets/RDRE' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-RDRE for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/RDRE'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_rdre_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Run Down Rate Export (RDRE) stream
    This function requests data from the '/datasets/RDRE/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-RDRE-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/RDRE/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ruri_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Run Up Rate Import (RURI)
    This function requests data from the '/datasets/RURI' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-RURI for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/RURI'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ruri_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Run Up Rate Import (RURI) stream
    This function requests data from the '/datasets/RURI/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-RURI-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/RURI/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_rdri_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Run Down Rate Import (RDRI)
    This function requests data from the '/datasets/RDRI' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-RDRI for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/RDRI'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_rdri_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Run Down Rate Import (RDRI) stream
    This function requests data from the '/datasets/RDRI/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-RDRI-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/RDRI/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_bid_offer_bmunit_bmunit_from_from_to_to(bmUnit, date_from, date_to, settlementPeriodFrom, settlementPeriodTo, json):

    """
    Get the Bid-offer data per BMU (BOD)
    This function requests data from the '/balancing/bid-offer' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-bid-offer for more information

    Args:
        bmUnit: The BM Unit date_to query.
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/bid-offer'

    params = {
        'bmUnit': bmUnit,
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_acceptances_bmunit_bmunit_from_from_to_to(bmUnit, date_from, date_to, settlementPeriodFrom, settlementPeriodTo, json):

    """
    Get the Bid-offer acceptances per BMU (BOALF)
    This function requests data from the '/balancing/acceptances' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-acceptances for more information

    Args:
        bmUnit: The BM Unit date_to query.
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/acceptances'

    params = {
        'bmUnit': bmUnit,
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_nonbm_netbsad_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, includeZero, json):

    """
    Get the Net balancing services adjustment time series (NETBSAD)
    This function requests data from the '/balancing/nonbm/netbsad' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-nonbm-netbsad for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        includeZero: Include data points with a generation of zero.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/nonbm/netbsad'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'includeZero': includeZero,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_nonbm_netbsad_events_count_count(count, before, settlementPeriodBefore, json):

    """
    Get the Net balancing services adjustment events (NETBSAD)
    This function requests data from the '/balancing/nonbm/netbsad/events' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-nonbm-netbsad-events for more information

    Args:
        count: Format - int32. The number of events date_to return.
        before: Format - date-time (as date-time in RFC3339). If specified, filters events date_to those with a start time before or at the date, or a settlement date before the date if
settlementPeriodBefore is also specified.
If omitted, latest events are returned.
        settlementPeriodBefore: Format - int32. Filters events date_to those with a settlement period before or at the value.
Before parameter must be specified if this is specified.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/nonbm/netbsad/events'

    params = {
        'count': count,
        'before': before,
        'settlementPeriodBefore': settlementPeriodBefore,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_netbsad_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, json):

    """
    Get the Net Balancing Services Adjustment Data (NETBSAD)
    This function requests data from the '/datasets/NETBSAD' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NETBSAD for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NETBSAD'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_netbsad_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo):

    """
    Get the Net Balancing Services Adjustment Data (NETBSAD)
    This function requests data from the '/datasets/NETBSAD/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-NETBSAD-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/NETBSAD/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_fuelhh_stream(publishDateTimeFrom, publishDateTimeTo, settlementDateFrom, settlementDateTo, settlementPeriod, fuelType):

    """
    Get the Half-hourly generation outturn by fuel type (FUELHH) stream
    This function requests data from the '/datasets/FUELHH/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-FUELHH-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        settlementDateFrom: Format - date (as full-date in RFC3339).
        settlementDateTo: Format - date (as full-date in RFC3339).
        settlementPeriod: List of Settlement Periods
        fuelType: Fuel Type e.g. NUCLEAR

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/FUELHH/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'settlementDateFrom': settlementDateFrom,
        'settlementDateTo': settlementDateTo,
        'settlementPeriod': settlementPeriod,
        'fuelType': fuelType,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_fuelinst_stream(publishDateTimeFrom, publishDateTimeTo, settlementDateFrom, settlementDateTo, settlementPeriod, fuelType):

    """
    Get the Instantaneous generation outturn by fuel type (FUELINST) stream
    This function requests data from the '/datasets/FUELINST/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-FUELINST-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        settlementDateFrom: Format - date (as full-date in RFC3339).
        settlementDateTo: Format - date (as full-date in RFC3339).
        settlementPeriod: List of Settlement Periods
        fuelType: Fuel Type e.g. NUCLEAR

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/FUELINST/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'settlementDateFrom': settlementDateFrom,
        'settlementDateTo': settlementDateTo,
        'settlementPeriod': settlementPeriod,
        'fuelType': fuelType,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_mels_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Maximum Export Limit (MELS)
    This function requests data from the '/datasets/MELS' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MELS for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MELS'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_mils_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Maximum Import Limit (MILS)
    This function requests data from the '/datasets/MILS' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MILS for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MILS'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_disbsad_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, json):

    """
    Get the Disaggregated Balancing Services Adjustment Data (DISBSAD)
    This function requests data from the '/datasets/DISBSAD' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-DISBSAD for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/DISBSAD'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_disbsad_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo):

    """
    Get the Disaggregated Balancing Services Adjustment Data (DISBSAD) stream
    This function requests data from the '/datasets/DISBSAD/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-DISBSAD-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/DISBSAD/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_bod_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Bid Offer Data (BOD)
    This function requests data from the '/datasets/BOD' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-BOD for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/BOD'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_bod_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Bid-Offer Data (BOD) stream
    This function requests data from the '/datasets/BOD/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-BOD-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/BOD/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_boalf_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit, json):

    """
    Get the Bid Offer Acceptance Level Flagged (BOALF)
    This function requests data from the '/datasets/BOALF' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-BOALF for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/BOALF'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_boalf_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Bid Offer Acceptance Level Flagged (BOALF) stream
    This function requests data from the '/datasets/BOALF/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-BOALF-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/BOALF/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_nonbm_disbsad_details_settlementdate_settlementdate_settlement(settlementDate, settlementPeriod, json):

    """
    Get the Disaggregated balancing services adjustment per settlement period (DISBSAD)
    This function requests data from the '/balancing/nonbm/disbsad/details' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-nonbm-disbsad-details for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). The settlement date date_to query.
        settlementPeriod: Format - int32. The settlement period date_to query. This should be an integer date_from 1-50 inclusive.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/nonbm/disbsad/details'

    params = {
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_pricing_market_index_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, dataProviders, json):

    """
    Get the Market Index Data (MID) price time series
    This function requests data from the '/balancing/pricing/market-index' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-pricing-market-index for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        dataProviders: The data providers date_to query. If no data provider is selected both will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/pricing/market-index'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'dataProviders': dataProviders,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_peak_triad_data_data(data, triadSeasonStartYear, json):

    """
    Get the Triad demand peaks (S0142, ITSDO, FUELHH)
    This function requests data from the '/demand/peak/triad' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-peak-triad for more information

    Args:
        data: The type of data. Supports values of 'operational' or 'settlement'. Use operational, settlement.
        triadSeasonStartYear: Format - int32. A year indicating the Triad season starting on 1 November of the given year.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/peak/triad'

    params = {
        'data': data,
        'triadSeasonStartYear': triadSeasonStartYear,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_pn_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Physical Notifications (PN) stream
    This function requests data from the '/datasets/PN/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-PN-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The settlement date date_to query date_from.
        date_to: Format - date-time (as date-time in RFC3339). The settlement date date_to query up date_to.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/PN/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_qpn_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Quiescent Physical Notifications (QPN) stream
    This function requests data from the '/datasets/QPN/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-QPN-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The settlement date date_to query date_from.
        date_to: Format - date-time (as date-time in RFC3339). The settlement date date_to query up date_to.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/QPN/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_lolpdrm_publishdatetimefrom_publishdatetimefrom_publishdatetime(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Loss of load probability and de-rated margin (LOLPDRM)
    This function requests data from the '/datasets/LOLPDRM' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-LOLPDRM for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/LOLPDRM'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_lolpdrm_stream_publishdatetimefrom_publishdatetimefrom_publishd(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Loss of load probability and de-rated margin (LOLPDRM)
    This function requests data from the '/datasets/LOLPDRM/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-LOLPDRM-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/LOLPDRM/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_remit_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the The Regulation on Wholesale Energy Markets Integrity and Transparency (REMIT)
    This function requests data from the '/datasets/REMIT' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-REMIT for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/REMIT'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_remit_stream_publishdatetimefrom_publishdatetimefrom_publishdat(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the The Regulation on Wholesale Energy Markets Integrity and Transparency (REMIT) stream
    This function requests data from the '/datasets/REMIT/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-REMIT-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/REMIT/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_cdn_publishdatetimefrom_publishdatetimefrom_publishdatetimeto_p(publishDateTimeFrom, publishDateTimeTo, bscPartyId, json):

    """
    Get the Credit default notices (CDN)
    This function requests data from the '/datasets/CDN' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-CDN for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        bscPartyId:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/CDN'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'bscPartyId': bscPartyId,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_cdn_stream_publishdatetimefrom_publishdatetimefrom_publishdatet(publishDateTimeFrom, publishDateTimeTo, bscPartyId):

    """
    Get the Credit default notices (CDN)
    This function requests data from the '/datasets/CDN/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-CDN-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        bscPartyId:

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/CDN/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'bscPartyId': bscPartyId,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_remit_messageid_messageid(messageId, json):

    """
    Get the Bulk fetch message details by IDs
    This function requests data from the '/remit' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-remit for more information

    Args:
        messageId:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/remit'

    params = {
        'messageId': messageId,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_remit_messageid(messageId, json):

    """
    Get the Fetch message details by ID
    This function requests data from the '/remit/{messageId}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-remit-{messageId} for more information

    Args:
        messageId: Format - int32.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/remit/{messageId}'

    params = {
        'messageId': messageId,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_remit_search_mrid_mrid(mrid, revisionNumber, json):

    """
    Get the Fetch message details by mRID
    This function requests data from the '/remit/search' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-remit-search for more information

    Args:
        mrid:
        revisionNumber: Format - int32.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/remit/search'

    params = {
        'mrid': mrid,
        'revisionNumber': revisionNumber,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_remit_revisions(mrid, messageId, json):

    """
    Get the List all message revisions
    This function requests data from the '/remit/revisions' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-remit-revisions for more information

    Args:
        mrid:
        messageId: Format - int32.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/remit/revisions'

    params = {
        'mrid': mrid,
        'messageId': messageId,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_reference_remit_participants_all():

    """
    Get the Participants
    This function requests data from the '/reference/remit/participants/all' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-reference-remit-participants-all for more information

    Args:

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/reference/remit/participants/all'

    params = {
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_reference_remit_assets_all():

    """
    Get the Assets
    This function requests data from the '/reference/remit/assets/all' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-reference-remit-assets-all for more information

    Args:

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/reference/remit/assets/all'

    params = {
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_reference_remit_fueltypes_all():

    """
    Get the REMIT fuel types
    This function requests data from the '/reference/remit/fueltypes/all' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-reference-remit-fueltypes-all for more information

    Args:

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/reference/remit/fueltypes/all'

    params = {
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_remit_list_by_publish_from_from_to_to(date_from, date_to, participantId, assetId, messageType, unavailabilityType, eventType, fuelType, latestRevisionOnly, profileOnly, json):

    """
    Get the List messages by publish time
    This function requests data from the '/remit/list/by-publish' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-remit-list-by-publish for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        participantId:
        assetId:
        messageType:
        unavailabilityType:
        eventType:
        fuelType:
        latestRevisionOnly:
        profileOnly:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/remit/list/by-publish'

    params = {
        'from': date_from,
        'to': date_to,
        'participantId': participantId,
        'assetId': assetId,
        'messageType': messageType,
        'unavailabilityType': unavailabilityType,
        'eventType': eventType,
        'fuelType': fuelType,
        'latestRevisionOnly': latestRevisionOnly,
        'profileOnly': profileOnly,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_remit_list_by_publish_stream_from_from_to_to(date_from, date_to, participantId, assetId, messageType, unavailabilityType, eventType, fuelType, latestRevisionOnly, profileOnly):

    """
    Get the List messages by publish time (stream)
    This function requests data from the '/remit/list/by-publish/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-remit-list-by-publish-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        participantId:
        assetId:
        messageType:
        unavailabilityType:
        eventType:
        fuelType:
        latestRevisionOnly:
        profileOnly:

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/remit/list/by-publish/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'participantId': participantId,
        'assetId': assetId,
        'messageType': messageType,
        'unavailabilityType': unavailabilityType,
        'eventType': eventType,
        'fuelType': fuelType,
        'latestRevisionOnly': latestRevisionOnly,
        'profileOnly': profileOnly,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_remit_list_by_event_from_from_to_to(date_from, date_to, participantId, assetId, messageType, unavailabilityType, eventType, fuelType, latestRevisionOnly, profileOnly, json):

    """
    Get the List messages by event time
    This function requests data from the '/remit/list/by-event' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-remit-list-by-event for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        participantId:
        assetId:
        messageType:
        unavailabilityType:
        eventType:
        fuelType:
        latestRevisionOnly:
        profileOnly:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/remit/list/by-event'

    params = {
        'from': date_from,
        'to': date_to,
        'participantId': participantId,
        'assetId': assetId,
        'messageType': messageType,
        'unavailabilityType': unavailabilityType,
        'eventType': eventType,
        'fuelType': fuelType,
        'latestRevisionOnly': latestRevisionOnly,
        'profileOnly': profileOnly,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_remit_list_by_event_stream_from_from_to_to(date_from, date_to, participantId, assetId, messageType, unavailabilityType, eventType, fuelType, latestRevisionOnly, profileOnly):

    """
    Get the List messages by event time (stream)
    This function requests data from the '/remit/list/by-event/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-remit-list-by-event-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        participantId:
        assetId:
        messageType:
        unavailabilityType:
        eventType:
        fuelType:
        latestRevisionOnly:
        profileOnly:

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/remit/list/by-event/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'participantId': participantId,
        'assetId': assetId,
        'messageType': messageType,
        'unavailabilityType': unavailabilityType,
        'eventType': eventType,
        'fuelType': fuelType,
        'latestRevisionOnly': latestRevisionOnly,
        'profileOnly': profileOnly,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_cdn(json):

    """
    Get the [DEPRECATED] Credit default notices (CDN)
    This function requests data from the '/CDN' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-CDN for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/CDN'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_syswarn(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the System warnings (SYSWARN)
    This function requests data from the '/datasets/SYSWARN' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-SYSWARN for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/SYSWARN'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_syswarn_stream(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the System warnings (SYSWARN) stream
    This function requests data from the '/datasets/SYSWARN/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-SYSWARN-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/SYSWARN/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_system_warnings(warningType, publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the System warnings (SYSWARN)
    This function requests data from the '/system/warnings' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-system-warnings for more information

    Args:
        warningType:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/system/warnings'

    params = {
        'warningType': warningType,
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_dci(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Demand control instructions (DCI)
    This function requests data from the '/datasets/DCI' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-DCI for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/DCI'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_dci_stream(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Demand control instructions (DCI) stream
    This function requests data from the '/datasets/DCI/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-DCI-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/DCI/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_generation_actual_per_type_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, json):

    """
    Get the Historic actual generation automatically down-sampled (AGPT/B1620)
    This function requests data from the '/generation/actual/per-type' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-generation-actual-per-type for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        settlementPeriodFrom: Format - int32.
        settlementPeriodTo: Format - int32.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/generation/actual/per-type'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_generation_actual_per_type_wind_and_solar_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, json):

    """
    Get the Historic actual or estimated wind and solar power generation (AGWS/B1630)
    This function requests data from the '/generation/actual/per-type/wind-and-solar' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-generation-actual-per-type-wind-and-solar for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        settlementPeriodFrom: Format - int32.
        settlementPeriodTo: Format - int32.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/generation/actual/per-type/wind-and-solar'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_agws_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Actual Or Estimated Wind And Solar Power Generation (AGWS / B1630)
    This function requests data from the '/datasets/AGWS' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-AGWS for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/AGWS'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_agws_stream_publishdatetimefrom_publishdatetimefrom_publishdate(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Actual Or Estimated Wind And Solar Power Generation (AGWS / B1630) stream
    This function requests data from the '/datasets/AGWS/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-AGWS-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/AGWS/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_agpt_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Actual Aggregated Generation Per Type (AGPT / B1620)
    This function requests data from the '/datasets/AGPT' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-AGPT for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/AGPT'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_agpt_stream_publishdatetimefrom_publishdatetimefrom_publishdate(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Actual Aggregated Generation Per Type (AGPT / B1620) stream
    This function requests data from the '/datasets/AGPT/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-AGPT-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/AGPT/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_atl_publishdatetimefrom_publishdatetimefrom_publishdatetimeto_p(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Actual Total Load Per Bidding Zone (ATL / B0610)
    This function requests data from the '/datasets/ATL' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-ATL for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/ATL'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_atl_stream_publishdatetimefrom_publishdatetimefrom_publishdatet(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Actual Total Load Per Bidding Zone (ATL / B0610) stream
    This function requests data from the '/datasets/ATL/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-ATL-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/ATL/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_datl_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Day-Ahead Total Load Forecast Per Bidding Zone (DATL / B0620)
    This function requests data from the '/datasets/DATL' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-DATL for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/DATL'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_datl_stream_publishdatetimefrom_publishdatetimefrom_publishdate(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Day-Ahead Total Load Forecast Per Bidding Zone (DATL / B0620) stream
    This function requests data from the '/datasets/DATL/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-DATL-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/DATL/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_watl_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Week-Ahead Total Load Forecast Per Bidding Zone (WATL / B0630)
    This function requests data from the '/datasets/WATL' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-WATL for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/WATL'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_watl_stream_publishdatetimefrom_publishdatetimefrom_publishdate(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Week-Ahead Total Load Forecast Per Bidding Zone (WATL / B0630) stream
    This function requests data from the '/datasets/WATL/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-WATL-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/WATL/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_system_demand_control_instructions(date_from, date_to, json):

    """
    Get the Demand control instructions (DCI)
    This function requests data from the '/system/demand-control-instructions' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-system-demand-control-instructions for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/system/demand-control-instructions'

    params = {
        'from': date_from,
        'to': date_to,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_total_day_ahead_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, json):

    """
    Get the Day-ahead total load forecast (DATL/B0620)
    This function requests data from the '/forecast/demand/total/day-ahead' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-total-day-ahead for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        settlementPeriodFrom: Format - int32.
        settlementPeriodTo: Format - int32.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/total/day-ahead'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_matl_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Month-Ahead Total Load Forecast Per Bidding Zone (MATL / B0640)
    This function requests data from the '/datasets/MATL' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MATL for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MATL'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_matl_stream_publishdatetimefrom_publishdatetimefrom_publishdate(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Month-Ahead Total Load Forecast Per Bidding Zone (MATL / B0640) stream
    This function requests data from the '/datasets/MATL/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MATL-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MATL/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_dgws_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Day-Ahead Generation For Wind And Solar (DGWS / B1440)
    This function requests data from the '/datasets/DGWS' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-DGWS for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/DGWS'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_dgws_stream_publishdatetimefrom_publishdatetimefrom_publishdate(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Day-Ahead Generation For Wind And Solar (DGWS / B1440) stream
    This function requests data from the '/datasets/DGWS/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-DGWS-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/DGWS/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_dag_publishdatetimefrom_publishdatetimefrom_publishdatetimeto_p(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Day-Ahead Aggregated Generation (DAG / B1430)
    This function requests data from the '/datasets/DAG' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-DAG for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/DAG'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_dag_stream_publishdatetimefrom_publishdatetimefrom_publishdatet(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Day-Ahead Aggregated Generation (DAG / B1430) stream
    This function requests data from the '/datasets/DAG/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-DAG-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/DAG/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_yatl_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Year-Ahead Total Load Forecast Per Bidding Zone (YATL / B0650)
    This function requests data from the '/datasets/YATL' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-YATL for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/YATL'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_yatl_stream_publishdatetimefrom_publishdatetimefrom_publishdate(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Year-Ahead Total Load Forecast Per Bidding Zone (YATL / B0650) stream
    This function requests data from the '/datasets/YATL/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-YATL-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/YATL/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ccm_publishdatetimefrom_publishdatetimefrom_publishdatetimeto_p(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Cost of Congestion Management (CCM / B1330)
    This function requests data from the '/datasets/CCM' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-CCM for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/CCM'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ccm_stream_publishdatetimefrom_publishdatetimefrom_publishdatet(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Cost of Congestion Management (CCM / B1330) stream
    This function requests data from the '/datasets/CCM/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-CCM-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/CCM/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_peak_indicative_data_data(data, triadSeasonStartYear, date_from, date_to, json):

    """
    Get the Indicative peak demand per day (S0142, ITSDO, FUELHH)
    This function requests data from the '/demand/peak/indicative' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-peak-indicative for more information

    Args:
        data: The type of data. Supports values of 'operational' or 'settlement'. Use operational, settlement.
        triadSeasonStartYear: Format - int32. A year indicating the Triad season starting on 1 November of the given year, e.g. 2021.
        date_from: Format - date (as full-date in RFC3339). The start of the requested date range.
        date_to: Format - date (as full-date in RFC3339). The end of the requested date range.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/peak/indicative'

    params = {
        'data': data,
        'triadSeasonStartYear': triadSeasonStartYear,
        'from': date_from,
        'to': date_to,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_generation_day_ahead_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, json):

    """
    Get the Day-ahead aggregated generation (DAG/B1430)
    This function requests data from the '/forecast/generation/day-ahead' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-generation-day-ahead for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        settlementPeriodFrom: Format - int32.
        settlementPeriodTo: Format - int32.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/generation/day-ahead'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_generation_wind_and_solar_day_ahead_from_from_to_to_processtype(date_from, date_to, processType, settlementPeriodFrom, settlementPeriodTo, json):

    """
    Get the Day-ahead generation forecast for wind and solar (DGWS/B1440)
    This function requests data from the '/forecast/generation/wind-and-solar/day-ahead' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-generation-wind-and-solar-day-ahead for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        processType:  Use all, day ahead, intraday process, intraday date_total.
        settlementPeriodFrom: Format - int32.
        settlementPeriodTo: Format - int32.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/generation/wind-and-solar/day-ahead'

    params = {
        'from': date_from,
        'to': date_to,
        'processType': processType,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_b1610_settlementdate_settlementdate_settlementperiod_settlement(settlementDate, settlementPeriod, bmUnit, json):

    """
    Get the Actual Generation Output Per Generation Unit (B1610)
    This function requests data from the '/datasets/B1610' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-B1610 for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339).
        settlementPeriod: Format - int32.
        bmUnit:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/B1610'

    params = {
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_b1610_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Actual Generation Output Per Generation Unit (B1610) stream
    This function requests data from the '/datasets/B1610/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-B1610-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit:

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/B1610/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_yafm_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Year Ahead Forecast Margin (YAFM / B0810)
    This function requests data from the '/datasets/YAFM' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-YAFM for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/YAFM'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_yafm_stream_publishdatetimefrom_publishdatetimefrom_publishdate(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Year Ahead Forecast Margin (YAFM / B0810) stream
    This function requests data from the '/datasets/YAFM/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-YAFM-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/YAFM/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_total_week_ahead_from_from_to_to(date_from, date_to, json):

    """
    Get the Week-ahead total load forecast (WATL/B0630)
    This function requests data from the '/forecast/demand/total/week-ahead' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-total-week-ahead for more information

    Args:
        date_from: Format - date (as full-date in RFC3339). The earliest forecast date date_to include.
        date_to: Format - date (as full-date in RFC3339). The latest forecast date date_to include.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/total/week-ahead'

    params = {
        'from': date_from,
        'to': date_to,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_demand_total_week_ahead_latest(json):

    """
    Get the Latest week-ahead total load forecast (WATL/B0630)
    This function requests data from the '/forecast/demand/total/week-ahead/latest' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-demand-total-week-ahead-latest for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/total/week-ahead/latest'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_generation_actual_per_type_day_total(json):

    """
    Get the Current snapshot of actual generation by fuel type categories (AGPT/B1620)
    This function requests data from the '/generation/actual/per-type/day-total' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-generation-actual-per-type-day-total for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/generation/actual/per-type/day-total'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_igca_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Installed Generation Capacity Aggregated (IGCA / B1410)
    This function requests data from the '/datasets/IGCA' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-IGCA for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/IGCA'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_igca_stream_publishdatetimefrom_publishdatetimefrom_publishdate(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Installed Generation Capacity Aggregated (IGCA / B1410) stream
    This function requests data from the '/datasets/IGCA/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-IGCA-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/IGCA/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_abuc_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Amount Of Balancing Reserves Under Contract (ABUC / B1720)
    This function requests data from the '/datasets/ABUC' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-ABUC for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/ABUC'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_abuc_stream_publishdatetimefrom_publishdatetimefrom_publishdate(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Amount Of Balancing Reserves Under Contract (ABUC / B1720) stream
    This function requests data from the '/datasets/ABUC/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-ABUC-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/ABUC/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_indod_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Initial National Demand outturn daily (INDOD)
    This function requests data from the '/datasets/INDOD' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-INDOD for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/INDOD'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_indod_stream_publishdatetimefrom_publishdatetimefrom_publishdat(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Initial National Demand outturn daily (INDOD) stream
    This function requests data from the '/datasets/INDOD/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-INDOD-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/INDOD/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_outturn_daily(settlementDateFrom, settlementDateTo, json):

    """
    Get the Initial National Demand outturn per day (INDOD)
    This function requests data from the '/demand/outturn/daily' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-outturn-daily for more information

    Args:
        settlementDateFrom: Format - date (as full-date in RFC3339).
        settlementDateTo: Format - date (as full-date in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/outturn/daily'

    params = {
        'settlementDateFrom': settlementDateFrom,
        'settlementDateTo': settlementDateTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_outturn_daily_stream(settlementDateFrom, settlementDateTo):

    """
    Get the Initial National Demand outturn per day (INDOD) stream
    This function requests data from the '/demand/outturn/daily/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-outturn-daily-stream for more information

    Args:
        settlementDateFrom: Format - date (as full-date in RFC3339).
        settlementDateTo: Format - date (as full-date in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/outturn/daily/stream'

    params = {
        'settlementDateFrom': settlementDateFrom,
        'settlementDateTo': settlementDateTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_peak_indicative_settlement_triadseason(triadSeason, json):

    """
    Get the Settlement data demand peaks for a Triad season (S0142)
    This function requests data from the '/demand/peak/indicative/settlement/{triadSeason}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-peak-indicative-settlement-{triadSeason} for more information

    Args:
        triadSeason: Format - int32. A year indicating the Triad season starting on 1 November of the given year
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/peak/indicative/settlement/{triadSeason}'

    params = {
        'triadSeason': triadSeason,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_peak_indicative_operational_triadseason(triadSeason, json):

    """
    Get the Operational data demand peaks for a Triad season (ITSDO, FUELHH)
    This function requests data from the '/demand/peak/indicative/operational/{triadSeason}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-peak-indicative-operational-{triadSeason} for more information

    Args:
        triadSeason: Format - int32. A year indicating the Triad season starting on 1 November of the given year
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/peak/indicative/operational/{triadSeason}'

    params = {
        'triadSeason': triadSeason,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_messages_settlementdate(settlementDate, json):

    """
    Get the Settlement messages by settlement date (SMSG)
    This function requests data from the '/balancing/settlement/messages/{settlementDate}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-messages-{settlementDate} for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). The settlement date date_to filter. This must be in the 'json' yyyy-MM-dd.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/messages/{settlementDate}'

    params = {
        'settlementDate': settlementDate,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_messages_settlementdate_settlementperiod(settlementPeriod, settlementDate, json):

    """
    Get the Settlement messages by settlement date and period (SMSG)
    This function requests data from the '/balancing/settlement/messages/{settlementDate}/{settlementPeriod}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-messages-{settlementDate}-{settlementPeriod} for more information

    Args:
        settlementPeriod: Format - int32. The settlement period date_to filter. This should be an integer date_from 1-50 inclusive.
        settlementDate: Format - date (as full-date in RFC3339). The settlement date date_to filter. This must be in the 'json' yyyy-MM-dd.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/messages/{settlementDate}/{settlementPeriod}'

    params = {
        'settlementPeriod': settlementPeriod,
        'settlementDate': settlementDate,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ppbr_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Prices Of Procured Balancing Reserves (PPBR / B1730)
    This function requests data from the '/datasets/PPBR' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-PPBR for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/PPBR'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_ppbr_stream_publishdatetimefrom_publishdatetimefrom_publishdate(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Prices Of Procured Balancing Reserves (PPBR / B1730) stream
    This function requests data from the '/datasets/PPBR/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-PPBR-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/PPBR/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_soso_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the SO-SO prices (SOSO)
    This function requests data from the '/datasets/SOSO' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-SOSO for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/SOSO'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_soso_stream_publishdatetimefrom_publishdatetimefrom_publishdate(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the SO-SO prices (SOSO) stream
    This function requests data from the '/datasets/SOSO/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-SOSO-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/SOSO/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_summary_settlementdate_settlementperiod(settlementDate, settlementPeriod, json):

    """
    Get the Settlement calculation summary (ISPSTACK, DISEBSP, MID, NETBSAD)
    This function requests data from the '/balancing/settlement/summary/{settlementDate}/{settlementPeriod}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-summary-{settlementDate}-{settlementPeriod} for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). This must be in the 'json' yyyy-MM-dd.
        settlementPeriod: Format - int32. This should be an integer date_from 1-50 inclusive.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/summary/{settlementDate}/{settlementPeriod}'

    params = {
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_soso_prices_from_from_to_to(date_from, date_to, json):

    """
    Get the SO-SO prices (SOSO)
    This function requests data from the '/soso/prices' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-soso-prices for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/soso/prices'

    params = {
        'from': date_from,
        'to': date_to,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_feib_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Financial Expenses and Income for Balancing (FEIB / B1790)
    This function requests data from the '/datasets/FEIB' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-FEIB for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/FEIB'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_feib_stream_publishdatetimefrom_publishdatetimefrom_publishdate(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Financial Expenses and Income for Balancing (FEIB / B1790) stream
    This function requests data from the '/datasets/FEIB/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-FEIB-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/FEIB/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_tudm_settlementdate_settlementdate_settlementperiod_settlementp(settlementDate, settlementPeriod, tradingUnitName, tradingUnitType, json):

    """
    Get the Trading unit data (S0491_TUDM)
    This function requests data from the '/datasets/TUDM' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-TUDM for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). The settlement date date_to filter. This must be in the 'json' yyyy-MM-dd.
        settlementPeriod: Format - int32.
        tradingUnitName:
        tradingUnitType:  Use Base, Sole, Others.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/TUDM'

    params = {
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'tradingUnitName': tradingUnitName,
        'tradingUnitType': tradingUnitType,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_tudm_stream_settlementdatefrom_settlementdatefrom_settlementper(settlementDateFrom, settlementPeriodFrom, settlementDateTo, settlementPeriodTo, tradingUnitName, tradingUnitType):

    """
    Get the Trading unit data (S0491_TUDM) stream
    This function requests data from the '/datasets/TUDM/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-TUDM-stream for more information

    Args:
        settlementDateFrom: Format - date (as full-date in RFC3339). The settlement date date_to filter date_from. This must be in the 'json' yyyy-MM-dd.
        settlementPeriodFrom: Format - int32.
        settlementDateTo: Format - date (as full-date in RFC3339). The settlement date date_to filter date_to. This must be in the 'json' yyyy-MM-dd.
        settlementPeriodTo: Format - int32.
        tradingUnitName:
        tradingUnitType:  Use Base, Sole, Others.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/TUDM/stream'

    params = {
        'settlementDateFrom': settlementDateFrom,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementDateTo': settlementDateTo,
        'settlementPeriodTo': settlementPeriodTo,
        'tradingUnitName': tradingUnitName,
        'tradingUnitType': tradingUnitType,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_stack_all_bidoffer_settlementdate_settlementperiod(bidOffer, settlementDate, settlementPeriod, json):

    """
    Get the Settlement bid-offer stacks by settlement period (ISPSTACK)
    This function requests data from the '/balancing/settlement/stack/all/{bidOffer}/{settlementDate}/{settlementPeriod}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-stack-all-{bidOffer}-{settlementDate}-{settlementPeriod} for more information

    Args:
        bidOffer: Filter if bid or offer data is date_to be returned. Use bid, offer.
        settlementDate: Format - date (as full-date in RFC3339). The settlement date for the filter. This must be in the 'json' yyyy-MM-dd.
        settlementPeriod: Format - int32. The settlement period for the filter.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/stack/all/{bidOffer}/{settlementDate}/{settlementPeriod}'

    params = {
        'bidOffer': bidOffer,
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_system_prices_settlementdate(settlementDate, json):

    """
    Get the Settlement system prices by settlement date (DISEBSP)
    This function requests data from the '/balancing/settlement/system-prices/{settlementDate}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-system-prices-{settlementDate} for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). The settlement date date_to filter. This must be in the 'json' yyyy-MM-dd.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/system-prices/{settlementDate}'

    params = {
        'settlementDate': settlementDate,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_system_prices_settlementdate_settlementperiod(settlementPeriod, settlementDate, json):

    """
    Get the Settlement system prices by settlement date and period (DISEBSP)
    This function requests data from the '/balancing/settlement/system-prices/{settlementDate}/{settlementPeriod}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-system-prices-{settlementDate}-{settlementPeriod} for more information

    Args:
        settlementPeriod: Format - int32. The settlement period date_to filter. This should be an integer date_from 1-50 inclusive.
        settlementDate: Format - date (as full-date in RFC3339). The settlement date date_to filter. This must be in the 'json' yyyy-MM-dd.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/system-prices/{settlementDate}/{settlementPeriod}'

    params = {
        'settlementPeriod': settlementPeriod,
        'settlementDate': settlementDate,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_acceptance_volumes_all_bidoffer_settlementdate(settlementDate, bidOffer, bmUnit, json):

    """
    Get the Acceptance volumes by settlement date (BOAV)
    This function requests data from the '/balancing/settlement/acceptance/volumes/all/{bidOffer}/{settlementDate}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-acceptance-volumes-all-{bidOffer}-{settlementDate} for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). This must be in the 'json' yyyy-MM-dd.
        bidOffer:  Use bid, offer.
        bmUnit: Elexon or NGC BMU IDs can be used. If omitted, results for all BM units will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/acceptance/volumes/all/{bidOffer}/{settlementDate}'

    params = {
        'settlementDate': settlementDate,
        'bidOffer': bidOffer,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_acceptance_volumes_all_bidoffer_settlementdate_sett(settlementPeriod, settlementDate, bidOffer, bmUnit, json):

    """
    Get the Acceptance volumes by settlement period (BOAV)
    This function requests data from the '/balancing/settlement/acceptance/volumes/all/{bidOffer}/{settlementDate}/{settlementPeriod}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-acceptance-volumes-all-{bidOffer}-{settlementDate}-{settlementPeriod} for more information

    Args:
        settlementPeriod: Format - int32. This should be an integer date_from 1-50 inclusive.
        settlementDate: Format - date (as full-date in RFC3339). This must be in the 'json' yyyy-MM-dd.
        bidOffer:  Use bid, offer.
        bmUnit: Elexon or NGC BMU IDs can be used. If omitted, results for all BM units will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/acceptance/volumes/all/{bidOffer}/{settlementDate}/{settlementPeriod}'

    params = {
        'settlementPeriod': settlementPeriod,
        'settlementDate': settlementDate,
        'bidOffer': bidOffer,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_indicative_volumes_all_bidoffer_settlementdate(settlementDate, bidOffer, bmUnit, json):

    """
    Get the Indicative volumes by settlement date (DISPTAV)
    This function requests data from the '/balancing/settlement/indicative/volumes/all/{bidOffer}/{settlementDate}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-indicative-volumes-all-{bidOffer}-{settlementDate} for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). This must be in the 'json' yyyy-MM-dd.
        bidOffer:  Use bid, offer.
        bmUnit: Elexon or NGC BMU IDs can be used. If omitted, results for all BM units will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/indicative/volumes/all/{bidOffer}/{settlementDate}'

    params = {
        'settlementDate': settlementDate,
        'bidOffer': bidOffer,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_indicative_volumes_all_bidoffer_settlementdate_sett(settlementPeriod, settlementDate, bidOffer, bmUnit, json):

    """
    Get the Indicative volumes by settlement period (DISPTAV)
    This function requests data from the '/balancing/settlement/indicative/volumes/all/{bidOffer}/{settlementDate}/{settlementPeriod}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-indicative-volumes-all-{bidOffer}-{settlementDate}-{settlementPeriod} for more information

    Args:
        settlementPeriod: Format - int32. This should be an integer date_from 1-50 inclusive.
        settlementDate: Format - date (as full-date in RFC3339). This must be in the 'json' yyyy-MM-dd.
        bidOffer:  Use bid, offer.
        bmUnit: Elexon or NGC BMU IDs can be used. If omitted, results for all BM units will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/indicative/volumes/all/{bidOffer}/{settlementDate}/{settlementPeriod}'

    params = {
        'settlementPeriod': settlementPeriod,
        'settlementDate': settlementDate,
        'bidOffer': bidOffer,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_indicative_cashflows_all_bidoffer_settlementdate(bidOffer, settlementDate, bmUnit, json):

    """
    Get the Indicative period BMU cashflows by settlement date (EBOCF)
    This function requests data from the '/balancing/settlement/indicative/cashflows/all/{bidOffer}/{settlementDate}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-indicative-cashflows-all-{bidOffer}-{settlementDate} for more information

    Args:
        bidOffer: Filter if bid or offer data is date_to be returned. Use bid, offer.
        settlementDate: Format - date (as full-date in RFC3339). The settlement date for the filter. This must be in the 'json' yyyy-MM-dd.
        bmUnit: The BM Units date_to query. Elexon or NGC BMU IDs can be used. If omitted, results for all BM units will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/indicative/cashflows/all/{bidOffer}/{settlementDate}'

    params = {
        'bidOffer': bidOffer,
        'settlementDate': settlementDate,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_indicative_cashflows_all_bidoffer_settlementdate_se(settlementPeriod, bidOffer, settlementDate, bmUnit, json):

    """
    Get the Indicative period BMU cashflows by settlement period (EBOCF)
    This function requests data from the '/balancing/settlement/indicative/cashflows/all/{bidOffer}/{settlementDate}/{settlementPeriod}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-indicative-cashflows-all-{bidOffer}-{settlementDate}-{settlementPeriod} for more information

    Args:
        settlementPeriod: Format - int32. The settlement period for the filter.
        bidOffer: Filter if bid or offer data is date_to be returned. Use bid, offer.
        settlementDate: Format - date (as full-date in RFC3339). The settlement date for the filter. This must be in the 'json' yyyy-MM-dd.
        bmUnit: The BM Units date_to query. Elexon or NGC BMU IDs can be used. If omitted, results for all BM units will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/indicative/cashflows/all/{bidOffer}/{settlementDate}/{settlementPeriod}'

    params = {
        'settlementPeriod': settlementPeriod,
        'bidOffer': bidOffer,
        'settlementDate': settlementDate,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_aobe_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Accepted Offered Balancing Energy (AOBE) (PREVIEW)
    This function requests data from the '/datasets/AOBE' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-AOBE for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/AOBE'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_aobe_stream_publishdatetimefrom_publishdatetimefrom_publishdate(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Accepted Offered Balancing Energy (AOBE) stream (PREVIEW)
    This function requests data from the '/datasets/AOBE/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-AOBE-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/AOBE/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_acceptances_all_latest(json):

    """
    Get the Latest market-wide bid-offer acceptances (BOALF)
    This function requests data from the '/balancing/acceptances/all/latest' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-acceptances-all-latest for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/acceptances/all/latest'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_outturn(settlementDateFrom, settlementDateTo, settlementPeriod, json):

    """
    Get the Initial National Demand outturn (INDO)
    This function requests data from the '/demand/outturn' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-outturn for more information

    Args:
        settlementDateFrom: Format - date (as full-date in RFC3339). The settlement date date_from for the filter. This must be in the 'json' yyyy-MM-dd.
        settlementDateTo: Format - date (as full-date in RFC3339). The settlement date date_to for the filter. This must be in the 'json' yyyy-MM-dd.
        settlementPeriod:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/outturn'

    params = {
        'settlementDateFrom': settlementDateFrom,
        'settlementDateTo': settlementDateTo,
        'settlementPeriod': settlementPeriod,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_outturn_stream(settlementDateFrom, settlementDateTo, settlementPeriod):

    """
    Get the Initial National Demand outturn (INDO) stream
    This function requests data from the '/demand/outturn/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-outturn-stream for more information

    Args:
        settlementDateFrom: Format - date (as full-date in RFC3339). The settlement date date_from for the filter. This must be in the 'json' yyyy-MM-dd.
        settlementDateTo: Format - date (as full-date in RFC3339). The settlement date date_to for the filter. This must be in the 'json' yyyy-MM-dd.
        settlementPeriod:

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/outturn/stream'

    params = {
        'settlementDateFrom': settlementDateFrom,
        'settlementDateTo': settlementDateTo,
        'settlementPeriod': settlementPeriod,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_outturn_summary(date_from, date_to, json):

    """
    Get the System demand summary (FUELINST)
    This function requests data from the '/demand/outturn/summary' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-outturn-summary for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/outturn/summary'

    params = {
        'from': date_from,
        'to': date_to,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_total_actual(json):

    """
    Get the This endpoint is obsolete, and this location may be removed with no further notice.
    This function requests data from the '/demand/total/actual' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-total-actual for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/total/actual'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_demand_actual_total_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, json):

    """
    Get the Actual total load (ATL/B0610)
    This function requests data from the '/demand/actual/total' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-demand-actual-total for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        settlementPeriodFrom: Format - int32.
        settlementPeriodTo: Format - int32.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/demand/actual/total'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_generation_outturn(date_from, date_to, json):

    """
    Get the Total generation outturn (FUELINST)
    This function requests data from the '/generation/outturn' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-generation-outturn for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/generation/outturn'

    params = {
        'from': date_from,
        'to': date_to,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_generation_outturn_interconnectors(publishDateTimeFrom, publishDateTimeTo, settlementDateFrom, settlementDateTo, settlementPeriod, interconnecdate_torName, json):

    """
    Get the Historic half-hourly interconnector flows (FUELINST)
    This function requests data from the '/generation/outturn/interconnectors' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-generation-outturn-interconnectors for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        settlementDateFrom: Format - date (as full-date in RFC3339). The settlement date date_from filter. This must be in the 'json' yyyy-MM-dd.
        settlementDateTo: Format - date (as full-date in RFC3339). The settlement date date_to filter. This must be in the 'json' yyyy-MM-dd.
        settlementPeriod:
        interconnecdate_torName:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/generation/outturn/interconnectors'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'settlementDateFrom': settlementDateFrom,
        'settlementDateTo': settlementDateTo,
        'settlementPeriod': settlementPeriod,
        'interconnectorName': interconnecdate_torName,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_generation_outturn_current(fuelType, json):

    """
    Get the Current snapshot of generation by fuel type categories (FUELINST, FUELHH)
    This function requests data from the '/generation/outturn/current' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-generation-outturn-current for more information

    Args:
        fuelType:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/generation/outturn/current'

    params = {
        'fuelType': fuelType,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_availability_summary_14d(json):

    """
    Get the Down-sampled fourteen-day generation forecast (FOU2T14D)
    This function requests data from the '/forecast/availability/summary/14D' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-availability-summary-14D for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/availability/summary/14D'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_availability_summary_3yw(json):

    """
    Get the Down-sampled three-year generation forecast (FOU2T3YW)
    This function requests data from the '/forecast/availability/summary/3YW' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-availability-summary-3YW for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/availability/summary/3YW'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_lolpdrm_forecast_evolution(json):

    """
    Get the Loss of load probability and de-rated margin forecast (LOLPDRM)
    This function requests data from the '/lolpdrm/forecast/evolution' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-lolpdrm-forecast-evolution for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/lolpdrm/forecast/evolution'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_default_notices(json):

    """
    Get the Default notices (CDN)
    This function requests data from the '/balancing/settlement/default-notices' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-default-notices for more information

    Args:
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/default-notices'

    params = {
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_acceptances_all_settlementdate_settlementperiod(settlementDate, settlementPeriod, json):

    """
    Get the Historic acceptances by settlement period (ISPSTACK, BOALF, BOD)
    This function requests data from the '/balancing/settlement/acceptances/all/{settlementDate}/{settlementPeriod}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-acceptances-all-{settlementDate}-{settlementPeriod} for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). The settlement date for the filter. This must be in the 'json' yyyy-MM-dd.
        settlementPeriod: Format - int32. The settlement period date_to filter. This should be an integer date_from 1-50 inclusive.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/acceptances/all/{settlementDate}/{settlementPeriod}'

    params = {
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_forecast_system_loss_of_load_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, json):

    """
    Get the Loss of load probability and de-rated margin forecast (LOLPDRM)
    This function requests data from the '/forecast/system/loss-of-load' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-forecast-system-loss-of-load for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339).
        date_to: Format - date-time (as date-time in RFC3339).
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/forecast/system/loss-of-load'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_dynamic_all_settlementdate_settlementdate_settlementperiod_set(settlementDate, settlementPeriod, bmUnit, dataset, json):

    """
    Get the Market-wide dynamic data (SEL, SIL, MZT, MNZT, MDV, MDP, NTB, NTO, NDZ)
    This function requests data from the '/balancing/dynamic/all' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-dynamic-all for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). The settlement date or datetime date_to filter.
        settlementPeriod: Format - int32. The settlement period date_to filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM Units date_to query. Elexon or NGC BMU IDs can be used. If omitted, results for all BM units will be returned.
        dataset: Datasets date_to filter. If omitted, all datasets will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/dynamic/all'

    params = {
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'bmUnit': bmUnit,
        'dataset': dataset,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_dynamic_rates_all_settlementdate_settlementdate_settlementperi(settlementDate, settlementPeriod, bmUnit, dataset, json):

    """
    Get the Market-wide rate data (RDRE, RURE, RDRI, RURI)
    This function requests data from the '/balancing/dynamic/rates/all' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-dynamic-rates-all for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). The settlement date date_to filter.
        settlementPeriod: Format - int32. The settlement period date_to filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM Units date_to query. Elexon or NGC BMU IDs can be used. If omitted, results for all BM units will be returned.
        dataset: Datasets date_to return. If omitted, all datasets will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/dynamic/rates/all'

    params = {
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'bmUnit': bmUnit,
        'dataset': dataset,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_physical_all_dataset_dataset_settlementdate_settlementdate_set(dataset, settlementDate, settlementPeriod, bmUnit, json):

    """
    Get the Market-wide physical data (PN, QPN, MILS, MELS)
    This function requests data from the '/balancing/physical/all' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-physical-all for more information

    Args:
        dataset: Dataset date_to query. Use PN, QPN, MILS, MELS.
        settlementDate: Format - date (as full-date in RFC3339). The settlement date for the filter.
        settlementPeriod: Format - int32. The settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM Units date_to query. Elexon or NGC BMU IDs can be used. If omitted, results for all BM units will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/physical/all'

    params = {
        'dataset': dataset,
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_bid_offer_all_settlementdate_settlementdate_settlementperiod_s(settlementDate, settlementPeriod, bmUnit, json):

    """
    Get the Market-wide bid-offer data (BOD)
    This function requests data from the '/balancing/bid-offer/all' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-bid-offer-all for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). The settlement date date_to filter.
        settlementPeriod: Format - int32. The settlement period date_to filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM Units date_to query. Elexon or NGC BMU IDs can be used. If omitted, results for all BM units will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/bid-offer/all'

    params = {
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_acceptances_all_settlementdate_settlementdate_settlementperiod(settlementDate, settlementPeriod, bmUnit, json):

    """
    Get the Market-wide bid-offer acceptances (BOALF)
    This function requests data from the '/balancing/acceptances/all' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-acceptances-all for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). The settlement date for the filter.
        settlementPeriod: Format - int32. The settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM Units date_to query. Elexon or NGC BMU IDs can be used. If omitted, results for all BM units will be returned.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/acceptances/all'

    params = {
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_pn_settlementdate_settlementdate_settlementperiod_settlementper(settlementDate, settlementPeriod, bmUnit, json):

    """
    Get the Physical Notifications (PN)
    This function requests data from the '/datasets/PN' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-PN for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). The settlement date date_to query. This must be in the 'json' yyyy-MM-dd.
        settlementPeriod: Format - int32. The settlement period date_to query. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/PN'

    params = {
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_qpn_settlementdate_settlementdate_settlementperiod_settlementpe(settlementDate, settlementPeriod, bmUnit, json):

    """
    Get the Quiescent Physical Notifications (QPN)
    This function requests data from the '/datasets/QPN' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-QPN for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). The settlement date date_to query. This must be in the 'json' yyyy-MM-dd.
        settlementPeriod: Format - int32. The settlement period date_to query. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/QPN'

    params = {
        'settlementDate': settlementDate,
        'settlementPeriod': settlementPeriod,
        'bmUnit': bmUnit,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_market_depth_settlementdate(settlementDate, json):

    """
    Get the Market depth by settlement date (IMBALNGC, BOD, DISEBSP, DISPTAV)
    This function requests data from the '/balancing/settlement/market-depth/{settlementDate}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-market-depth-{settlementDate} for more information

    Args:
        settlementDate: Format - date (as full-date in RFC3339). The settlement date for the filter. This must be in the 'json' yyyy-MM-dd.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/market-depth/{settlementDate}'

    params = {
        'settlementDate': settlementDate,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_balancing_settlement_market_depth_settlementdate_settlementperiod(settlementPeriod, settlementDate, json):

    """
    Get the Market depth by settlement period (IMBALNGC, BOD, DISEBSP, DISPTAV)
    This function requests data from the '/balancing/settlement/market-depth/{settlementDate}/{settlementPeriod}' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-balancing-settlement-market-depth-{settlementDate}-{settlementPeriod} for more information

    Args:
        settlementPeriod: Format - int32. The settlement period date_to filter. This should be an integer date_from 1-50 inclusive.
        settlementDate: Format - date (as full-date in RFC3339). The settlement date for the filter. This must be in the 'json' yyyy-MM-dd.
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/market-depth/{settlementDate}/{settlementPeriod}'

    params = {
        'settlementPeriod': settlementPeriod,
        'settlementDate': settlementDate,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_beb_publishdatetimefrom_publishdatetimefrom_publishdatetimeto_p(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Balancing Energy Bids (BEB) (PREVIEW)
    This function requests data from the '/datasets/BEB' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-BEB for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/BEB'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_beb_stream_publishdatetimefrom_publishdatetimefrom_publishdatet(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Balancing Energy Bids (BEB) stream (PREVIEW)
    This function requests data from the '/datasets/BEB/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-BEB-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/BEB/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_igcpu_publishdatetimefrom_publishdatetimefrom_publishdatetimeto(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Installed Generation Capacity per Unit (IGCPU / B1420)
    This function requests data from the '/datasets/IGCPU' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-IGCPU for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/IGCPU'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_igcpu_stream_publishdatetimefrom_publishdatetimefrom_publishdat(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Installed Generation Capacity per Unit (IGCPU / B1420) stream
    This function requests data from the '/datasets/IGCPU/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-IGCPU-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/IGCPU/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_cbs_publishdatetimefrom_publishdatetimefrom_publishdatetimeto_p(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Current Balancing State (CBS) (PREVIEW)
    This function requests data from the '/datasets/CBS' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-CBS for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/CBS'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_cbs_stream_publishdatetimefrom_publishdatetimefrom_publishdatet(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Current Balancing State (CBS) stream (PREVIEW)
    This function requests data from the '/datasets/CBS/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-CBS-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/CBS/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_pbc_publishdatetimefrom_publishdatetimefrom_publishdatetimeto_p(publishDateTimeFrom, publishDateTimeTo, json):

    """
    Get the Procured Balancing Capacity (PBC) (PREVIEW)
    This function requests data from the '/datasets/PBC' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-PBC for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).
        'json': Response data 'json'. Use json/xml date_to include metadata. Use json, xml, csv.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/PBC'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
        'format': 'json',
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_pbc_stream_publishdatetimefrom_publishdatetimefrom_publishdatet(publishDateTimeFrom, publishDateTimeTo):

    """
    Get the Procured Balancing Capacity (PBC) stream (PREVIEW)
    This function requests data from the '/datasets/PBC/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-PBC-stream for more information

    Args:
        publishDateTimeFrom: Format - date-time (as date-time in RFC3339).
        publishDateTimeTo: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/PBC/stream'

    params = {
        'publishDateTimeFrom': publishDateTimeFrom,
        'publishDateTimeTo': publishDateTimeTo,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_mels_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Maximum Export Limit (MELS) stream
    This function requests data from the '/datasets/MELS/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MELS-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MELS/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_datasets_mils_stream_from_from_to_to(date_from, date_to, settlementPeriodFrom, settlementPeriodTo, bmUnit):

    """
    Get the Maximum Import Limit (MILS) stream
    This function requests data from the '/datasets/MILS/stream' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-datasets-MILS-stream for more information

    Args:
        date_from: Format - date-time (as date-time in RFC3339). The "date_from" start time or settlement date for the filter.
        date_to: Format - date-time (as date-time in RFC3339). The "date_to" start time or settlement date for the filter.
        settlementPeriodFrom: Format - int32. The "date_from" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        settlementPeriodTo: Format - int32. The "date_to" settlement period for the filter. This should be an integer date_from 1-50 inclusive.
        bmUnit: The BM units date_to query. Add each unit separately. If no BM unit is selected all BM units will be displayed.

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/datasets/MILS/stream'

    params = {
        'from': date_from,
        'to': date_to,
        'settlementPeriodFrom': settlementPeriodFrom,
        'settlementPeriodTo': settlementPeriodTo,
        'bmUnit': bmUnit,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_interop_messagelistretrieval_eventstart_eventstart_eventend_eventend(eventStart, eventEnd):

    """
    Get the This endpoint is obsolete, and this location may be removed with no further notice. Use /remit/* or /datasets/REMIT endpoints instead.
    This function requests data from the '/interop/MessageListRetrieval' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-interop-MessageListRetrieval for more information

    Args:
        eventStart: Format - date-time (as date-time in RFC3339).
        eventEnd: Format - date-time (as date-time in RFC3339).

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/interop/MessageListRetrieval'

    params = {
        'eventStart': eventStart,
        'eventEnd': eventEnd,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df



def get_interop_messagedetailretrieval_messageid_messageid(messageId):

    """
    Get the This endpoint is obsolete, and this location may be removed with no further notice. Use /remit/* or /datasets/REMIT endpoints instead.
    This function requests data from the '/interop/MessageDetailRetrieval' endpoint
    See https://developer.data.elexon.co.uk/api-details#api=prod-insol-insights-api&operation=get-interop-MessageDetailRetrieval for more information

    Args:
        messageId:

    Raises:
        Exception: If the request fails or returns a non-200 status code.
    """

    url = 'https://data.elexon.co.uk/bmrs/api/v1/interop/MessageDetailRetrieval'

    params = {
        'messageId': messageId,
    }

    # Run the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Convert JSON output to DataFrame
        df = json_to_dataframe_with_metadata(data)
    else:
        raise Exception(f'Error: {response.status_code}')

    return df




#### Functions to retrieve data manually from API

In [259]:
def fetch_json(url):
    # Send GET request to the API
    response = requests.get(url)
    # Raise an error for bad status codes
    response.raise_for_status()
    # Parse the JSON response
    data = response.json()
    return data

def fetch_data(url):
    # Send GET request to the API
    response = requests.get(url)
    # Raise an error for bad status codes
    response.raise_for_status()

    # Parse the JSON response
    data = response.json()

    if isinstance(data, dict):
        # Extract the relevant part of the JSON
        records = data.get('data', [])
        metadata = data.get('metadata')

        # Convert the extracted data to DataFrames
        df = pd.DataFrame(records)

        if metadata is not None:
            df_meta = pd.DataFrame.from_dict(metadata, orient='index').T
        else:
            df_meta = "No metadata provided"
    elif isinstance(data, list):
        # If the data is a list, convert it directly to a DataFrame
        df = pd.DataFrame(data)
        df_meta = "No metadata provided"
    else:
        # Handle unexpected data types
        df = pd.DataFrame()
        df_meta = "No metadata provided"

    # Return the DataFrames
    return df, df_meta

def df_equal(df1, df2, sort_by):
    # Sort the DataFrames by the specified column
    df1 = df1.sort_values(by=sort_by).reset_index(drop=True)
    df2 = df2.sort_values(by=sort_by).reset_index(drop=True)

    # Align the columns of both DataFrames
    df1 = df1.sort_index(axis=1)
    df2 = df2.sort_index(axis=1)

    # Check if the sorted DataFrames are equal
    are_equal = df1.equals(df2)

    # If DataFrames are not equal, identify differing rows based on the specified column
    if not are_equal:
        # Compare DataFrames based on the specified column
        difference_mask = df1[sort_by] != df2[sort_by]

        # Extract differing rows
        diff_rows_df1 = df1[difference_mask]
        diff_rows_df2 = df2[difference_mask]
    else:
        diff_rows_df1 = pd.DataFrame()
        diff_rows_df2 = pd.DataFrame()

    return are_equal, diff_rows_df1, diff_rows_df2


### SECTION 2) Testing

The following sections are dedicated to testing the outputs of various functions generated using the generate_function_from_openapi function.


#### ENDPOINT 1: 'get_remit_list_by_event_from_from_to_to'

In [260]:
# Testing Functions
remit_list,remit_list_META = get_remit_list_by_event_from_from_to_to('2024-05-01T00:00:00Z', '2024-05-02T00:00:00Z',None, None, None, None, None, None,None, None, json)

# Data from API
remit_list_API, remit_list_API_meta = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/remit/list/by-event?from=2024-05-01T00:00:00Z&to=2024-05-02T00:00:00Z&format=json')

In [261]:
# JSON data
remit_list_json = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/remit/list/by-event?from=2024-05-01T00:00:00Z&to=2024-05-02T00:00:00Z&format=json')
remit_list_json

{'metadata': {'datasets': ['REMIT']},
 'data': [{'url': 'https://data.elexon.co.uk/bmrs/api/v1/remit/733547',
   'id': 733547,
   'mrid': '48X000000000277E-NGET-RMT-00004516',
   'revisionNumber': 2,
   'createdTime': '2024-05-02T10:00:05Z',
   'publishTime': '2024-05-02T09:01:11Z'},
  {'url': 'https://data.elexon.co.uk/bmrs/api/v1/remit/733544',
   'id': 733544,
   'mrid': '48X000000000277E-NGET-RMT-00004514',
   'revisionNumber': 2,
   'createdTime': '2024-05-02T09:51:06Z',
   'publishTime': '2024-05-02T08:52:10Z'},
  {'url': 'https://data.elexon.co.uk/bmrs/api/v1/remit/13723',
   'id': 13723,
   'mrid': '48X000000000283J-NGET-RMT-00032124',
   'revisionNumber': 2,
   'createdTime': '2023-06-30T15:20:56Z',
   'publishTime': '2023-06-30T14:27:26Z'},
  {'url': 'https://data.elexon.co.uk/bmrs/api/v1/remit/50988',
   'id': 50988,
   'mrid': '48X000000000283J-NGET-RMT-00034915',
   'revisionNumber': 2,
   'createdTime': '2024-01-12T16:36:54Z',
   'publishTime': '2024-01-12T16:37:21Z'},
  

In [262]:
# Test to see if metadata was retrieved correctly
remit_list_META

,datasets
0,REMIT


In [263]:
# test to see if the data was retrieved correctly
remit_list.head(1)

,id,mrid,revisionNumber,publishTime,url,createdTime
0,733547,48X000000000277E-NGET-RMT-00004516,2,2024-05-02T09:01:11Z,https://data.elexon.co.uk/bmrs/api/v1/remit/73...,2024-05-02T10:00:05Z


In [264]:
# Call function to check is dataframes are equal
are_equal, diff_rowsAPI, diff_rows_list = df_equal(remit_list_API, remit_list, 'id')

are_equal

True

#### ENDPOINT 2: get_balancing_acceptances_all_latest

In [265]:
# get data from script
balancing_acceptances_all_latest, balancing_acceptances_all_latest_Meta = get_balancing_acceptances_all_latest('json')

# Get data directly from API
balancing_acceptances_API,balancing_acceptances_API_Meta  = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/balancing/acceptances/all/latest?format=json')

In [266]:
balancing_acceptances_json  = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/balancing/acceptances/all/latest?format=json')
balancing_acceptances_json

{'metadata': {'datasets': ['BOALF']},
 'data': [{'settlementDate': '2024-06-03',
   'settlementPeriodFrom': 42,
   'settlementPeriodTo': 42,
   'timeFrom': '2024-06-03T19:54:00Z',
   'timeTo': '2024-06-03T19:55:00Z',
   'levelFrom': 100,
   'levelTo': 104,
   'nationalGridBmUnit': 'SGRWO-4',
   'bmUnit': 'T_SGRWO-4',
   'acceptanceNumber': 8055,
   'acceptanceTime': '2024-06-03T19:07:00Z',
   'deemedBoFlag': False,
   'soFlag': True,
   'storFlag': False,
   'rrFlag': False},
  {'settlementDate': '2024-06-03',
   'settlementPeriodFrom': 42,
   'settlementPeriodTo': 42,
   'timeFrom': '2024-06-03T19:53:00Z',
   'timeTo': '2024-06-03T19:54:00Z',
   'levelFrom': 50,
   'levelTo': 100,
   'nationalGridBmUnit': 'SGRWO-4',
   'bmUnit': 'T_SGRWO-4',
   'acceptanceNumber': 8055,
   'acceptanceTime': '2024-06-03T19:07:00Z',
   'deemedBoFlag': False,
   'soFlag': True,
   'storFlag': False,
   'rrFlag': False},
  {'settlementDate': '2024-06-03',
   'settlementPeriodFrom': 42,
   'settlementPerio

In [267]:
balancing_acceptances_all_latest.head(1)

,nationalGridBmUnit,levelTo,soFlag,timeTo,bmUnit,deemedBoFlag,acceptanceTime,levelFrom,storFlag,acceptanceNumber,settlementPeriodTo,rrFlag,settlementDate,timeFrom,settlementPeriodFrom
0,SGRWO-4,104,True,2024-06-03T19:55:00Z,T_SGRWO-4,False,2024-06-03T19:07:00Z,100,False,8055,42,False,2024-06-03,2024-06-03T19:54:00Z,42


In [268]:
balancing_acceptances_all_latest_Meta

,datasets
0,BOALF


In [269]:
# get a list of columns to compare
all_columns = balancing_acceptances_all_latest.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(balancing_acceptances_all_latest, balancing_acceptances_API,columns_to_compare)

are_equal

True

#### ENDPOINT 3: 'get_datasets_mils_stream_from_from_to_to'

In [270]:
# get data from script
balancing_acceptances,balancing_acceptances_META = get_datasets_mils_stream_from_from_to_to('2024-05-01T10:00:00Z', '2024-05-10T11:00:00Z',None, None,None)

# Get data directly from API
balancing_acceptances_API, balancing_acceptances_API_META = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/datasets/MILS/stream?from=2024-05-01T10:00:00Z&to=2024-05-10T11:00:00Z')


In [271]:
balancing_acceptances.head(1)

,dataset,settlementDate,settlementPeriod,timeFrom,timeTo,levelFrom,levelTo,notificationTime,notificationSequence,nationalGridBmUnit,bmUnit
0,MILS,2024-05-10,25,2024-05-10T11:00:00Z,2024-05-10T11:30:00Z,0,0,2024-03-20T10:00:36Z,123659077,IND-PWMR1,I_IND-PWMR1


In [272]:
balancing_acceptances_json = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/datasets/MILS?from=2024-05-01T10:00:00Z&to=2024-05-01T11:00:00Z&format=json')
balancing_acceptances_json

{'data': [{'dataset': 'MILS',
   'settlementDate': '2024-05-01',
   'settlementPeriod': 25,
   'timeFrom': '2024-05-01T11:00:00Z',
   'timeTo': '2024-05-01T11:30:00Z',
   'levelFrom': -37,
   'levelTo': -37,
   'notificationTime': '2024-05-01T10:04:02Z',
   'notificationSequence': 123183593,
   'nationalGridBmUnit': 'BURWB-1',
   'bmUnit': 'E_BURWB-1'},
  {'dataset': 'MILS',
   'settlementDate': '2024-05-01',
   'settlementPeriod': 25,
   'timeFrom': '2024-05-01T11:00:00Z',
   'timeTo': '2024-05-01T11:25:00Z',
   'levelFrom': -12,
   'levelTo': -12,
   'notificationTime': '2024-05-01T10:19:34Z',
   'notificationSequence': 123184054,
   'nationalGridBmUnit': 'AG-GEDF01',
   'bmUnit': '2__GLOND001'},
  {'dataset': 'MILS',
   'settlementDate': '2024-05-01',
   'settlementPeriod': 25,
   'timeFrom': '2024-05-01T11:00:00Z',
   'timeTo': '2024-05-01T11:30:00Z',
   'levelFrom': -40,
   'levelTo': -40,
   'notificationTime': '2024-05-01T10:00:38Z',
   'notificationSequence': 123183469,
   'nat

In [273]:
balancing_acceptances.head(1)

,dataset,settlementDate,settlementPeriod,timeFrom,timeTo,levelFrom,levelTo,notificationTime,notificationSequence,nationalGridBmUnit,bmUnit
0,MILS,2024-05-10,25,2024-05-10T11:00:00Z,2024-05-10T11:30:00Z,0,0,2024-03-20T10:00:36Z,123659077,IND-PWMR1,I_IND-PWMR1


In [274]:
balancing_acceptances_META

'No metadata provided'

In [275]:
balancing_acceptances_API.head(1)

,dataset,settlementDate,settlementPeriod,timeFrom,timeTo,levelFrom,levelTo,notificationTime,notificationSequence,nationalGridBmUnit,bmUnit
0,MILS,2024-05-10,25,2024-05-10T11:00:00Z,2024-05-10T11:30:00Z,0,0,2024-03-20T10:00:36Z,123659077,IND-PWMR1,I_IND-PWMR1


In [276]:
# get a list of columns to compare
all_columns = balancing_acceptances_API.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(balancing_acceptances, balancing_acceptances_API,columns_to_compare)

are_equal

True

#### ENDPOINT 4:  'get_forecast_surplus_weekly_evolution_year_year_week_week',

In [277]:
forecast_surplus_weekly, forecast_surplus_weekly_META= get_forecast_surplus_weekly_evolution_year_year_week_week(2024, 1, '52W', 'json')

# Get data directly from API
forecast_surplus_weekly_API,forecast_surplus_weekly_META = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/forecast/surplus/weekly/evolution?year=2024&week=1&range=52W&format=json')

In [278]:
forecast_surplus_weekly_json = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/forecast/surplus/weekly/evolution?year=2024&week=1&range=52W&format=json')
forecast_surplus_weekly_json

{'metadata': {'datasets': ['OCNMFW']},
 'data': [{'week': 1,
   'year': 2024,
   'weekStartDate': '2024-01-01',
   'publishTime': '2023-01-02T00:12:00Z',
   'surplus': 10777},
  {'week': 1,
   'year': 2024,
   'weekStartDate': '2024-01-01',
   'publishTime': '2023-01-02T01:11:00Z',
   'surplus': 10757},
  {'week': 1,
   'year': 2024,
   'weekStartDate': '2024-01-01',
   'publishTime': '2023-01-02T02:11:00Z',
   'surplus': 10757},
  {'week': 1,
   'year': 2024,
   'weekStartDate': '2024-01-01',
   'publishTime': '2023-01-02T03:11:00Z',
   'surplus': 10757},
  {'week': 1,
   'year': 2024,
   'weekStartDate': '2024-01-01',
   'publishTime': '2023-01-02T04:11:00Z',
   'surplus': 10757},
  {'week': 1,
   'year': 2024,
   'weekStartDate': '2024-01-01',
   'publishTime': '2023-01-02T05:12:00Z',
   'surplus': 10747},
  {'week': 1,
   'year': 2024,
   'weekStartDate': '2024-01-01',
   'publishTime': '2023-01-02T06:12:00Z',
   'surplus': 10747},
  {'week': 1,
   'year': 2024,
   'weekStartDate':

In [279]:
forecast_surplus_weekly_API.head(1)

,week,year,weekStartDate,publishTime,surplus
0,1,2024,2024-01-01,2023-01-02T00:12:00Z,10777


In [280]:
forecast_surplus_weekly_META

,datasets
0,OCNMFW


In [281]:
# get a list of columns to compare
all_columns = forecast_surplus_weekly.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(forecast_surplus_weekly,
                                                         forecast_surplus_weekly_API,
                                                         columns_to_compare)

are_equal

True

#### ENDPOINT 5: 'get_cdn',


In [282]:
Cdn,Cdn_META = get_cdn('json')

# Get data directly from API
Cdn_API, Cdn_API_META = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/CDN?format=json')

In [283]:
Cdn.head(1)

""
0


In [284]:
Cdn_META

'No metadata provided'

In [285]:
Cdn_API

""


In [286]:
# get a list of columns to compare
#all_columns = Cdn_API.columns
#columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
#are_equal, diff_rows_mockdata, diff_rows_list = df_equal(Cdn,
#                                                         Cdn_API,
#                                                         columns_to_compare)

print("No Data returned")

No Data returned


#### ENDPOINT 6: get_balancing_nonbm_disbsad_details_settlementdate_settlementdate_settlement

In [287]:
balancing_nonbm, balancing_nonbm_META= get_balancing_nonbm_disbsad_details_settlementdate_settlementdate_settlement('2024-05-01', 40, 'json')

# Get data directly from API
balancing_nonbm_API, balancing_nonbm_API_META = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/balancing/nonbm/disbsad/details?settlementDate=2024-05-01&settlementPeriod=40&format=json')

In [288]:
balancing_nonbm_json = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/balancing/nonbm/disbsad/details?settlementDate=2024-05-01&settlementPeriod=40&format=json')
balancing_nonbm_json

{'metadata': {'datasets': ['DISBSAD']},
 'data': [{'settlementDate': '2024-05-01',
   'settlementPeriod': 40,
   'startTime': '2024-05-01T18:30:00Z',
   'id': 1,
   'cost': 2501.33,
   'volume': 27.842,
   'price': 89.84,
   'soFlag': False,
   'storFlag': False,
   'partyId': 'ENWL',
   'assetId': 'OFR-ENWL-1',
   'isTendered': False,
   'service': 'Non-BM FR'},
  {'settlementDate': '2024-05-01',
   'settlementPeriod': 40,
   'startTime': '2024-05-01T18:30:00Z',
   'id': 2,
   'cost': 2475.0,
   'volume': 24.75,
   'price': 100.0,
   'soFlag': False,
   'storFlag': False,
   'partyId': 'UKPR',
   'assetId': 'OFR-UKPR-2',
   'isTendered': False,
   'service': 'Non-BM FR'}]}

In [289]:
balancing_nonbm_META

,datasets
0,DISBSAD


In [290]:
balancing_nonbm.head(1)

,isTendered,settlementPeriod,soFlag,cost,service,id,partyId,price,storFlag,assetId,settlementDate,volume,startTime
0,False,40,False,2501.33,Non-BM FR,1,ENWL,89.84,False,OFR-ENWL-1,2024-05-01,27.842,2024-05-01T18:30:00Z


In [291]:
# get a list of columns to compare
all_columns = balancing_nonbm_API.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(balancing_nonbm,
                                                         balancing_nonbm_API,
                                                         columns_to_compare)
are_equal

True

#### ENDPOINT 7:  get_datasets_atl_publishdatetimefrom_publishdatetimefrom_publishdatetimeto_p


In [292]:
datasets_atl, datasets_atl_META= get_datasets_atl_publishdatetimefrom_publishdatetimefrom_publishdatetimeto_p('2024-05-01T00:00:00Z', '2024-05-02T00:00:00Z', json)

# Get data directly from API
datasets_atl_API,datasets_atl_API_META = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/datasets/ATL?publishDateTimeFrom=2024-05-01T00:00:00Z&publishDateTimeTo=2024-05-02T00:00:00Z&format=json')

In [293]:
datasets_atl_json = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/datasets/ATL?publishDateTimeFrom=2024-05-01T00:00:00Z&publishDateTimeTo=2024-05-02T00:00:00Z&format=json')
datasets_atl_json

{'data': [{'dataset': 'ATL',
   'documentId': 'NGET-EMFIP-ATL-06439736',
   'documentRevisionNumber': 1,
   'publishTime': '2024-05-01T23:55:09Z',
   'startTime': '2024-05-01T22:00:00Z',
   'settlementDate': '2024-05-01',
   'settlementPeriod': 47,
   'quantity': 29924.0},
  {'dataset': 'ATL',
   'documentId': 'NGET-EMFIP-ATL-06439735',
   'documentRevisionNumber': 1,
   'publishTime': '2024-05-01T23:25:09Z',
   'startTime': '2024-05-01T21:30:00Z',
   'settlementDate': '2024-05-01',
   'settlementPeriod': 46,
   'quantity': 32118.0},
  {'dataset': 'ATL',
   'documentId': 'NGET-EMFIP-ATL-06439734',
   'documentRevisionNumber': 1,
   'publishTime': '2024-05-01T22:55:08Z',
   'startTime': '2024-05-01T21:00:00Z',
   'settlementDate': '2024-05-01',
   'settlementPeriod': 45,
   'quantity': 33697.0},
  {'dataset': 'ATL',
   'documentId': 'NGET-EMFIP-ATL-06439733',
   'documentRevisionNumber': 1,
   'publishTime': '2024-05-01T22:25:09Z',
   'startTime': '2024-05-01T20:30:00Z',
   'settlementD

In [294]:
datasets_atl.head(1)

,settlementPeriod,documentId,quantity,publishTime,settlementDate,startTime,documentRevisionNumber,dataset
0,47,NGET-EMFIP-ATL-06439736,29924.0,2024-05-01T23:55:09Z,2024-05-01,2024-05-01T22:00:00Z,1,ATL


In [295]:
datasets_atl_META

'No metadata provided'

In [296]:
# get a list of columns to compare
all_columns = datasets_atl_API.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(datasets_atl,
                                                         datasets_atl_API,
                                                         columns_to_compare)
are_equal

True

#### ENDPOINT 8: get_datasets_mdv_from_from_to_to


In [297]:
#datasets_mdv = get_datasets_mdv_from_from_to_to('2024-05-01T00:00:00Z', '2024-05-03T00:00:00Z', None, None, None, json)

# Get data directly from API
#datasets_mdv_API = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/datasets/MDV?from=2024-05-01T00:00:00Z&to=2024-05-03T00:00:00Z&format=json')

# get a list of columns to compare
#all_columns = datasets_mdv_API.columns
#columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
#are_equal, diff_rows_mockdata, diff_rows_list = df_equal(datasets_mdv, datasets_mdv_API,columns_to_compare)
#are_equal

print('No data')

No data


#### ENDPOINT 9: get_balancing_nonbm_stor_from_from_to_to


In [298]:
date_from = '2023-02-01T00:00:00Z'
date_to = '2023-02-02T00:00:00Z'

get_balancing_nonbm_stor_from_from_to_to(date_from, date_to, None, None, None, json)
print('no data')

no data


#### ENDPOINT 10: get_demand_peak


In [299]:
demand_peak,demand_peak_META = get_demand_peak('2024-05-01', '2024-05-07', json)

# Get data directly from API
demand_peak_API,demand_peak_API_META = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/demand/peak?from=2024-05-01&to=2024-05-07&format=json')

In [300]:
demand_peak_json = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/demand/peak?from=2024-05-01&to=2024-05-07&format=json')
demand_peak_json

{'metadata': {'datasets': ['ITSDO']},
 'data': [{'publishTime': '2024-05-01T18:30:00Z',
   'startTime': '2024-05-01T18:00:00Z',
   'settlementDate': '2024-05-01',
   'settlementPeriod': 39,
   'initialTransmissionSystemDemandOutturn': 32354},
  {'publishTime': '2024-05-02T18:30:00Z',
   'startTime': '2024-05-02T18:00:00Z',
   'settlementDate': '2024-05-02',
   'settlementPeriod': 39,
   'initialTransmissionSystemDemandOutturn': 32520},
  {'publishTime': '2024-05-03T08:30:00Z',
   'startTime': '2024-05-03T08:00:00Z',
   'settlementDate': '2024-05-03',
   'settlementPeriod': 19,
   'initialTransmissionSystemDemandOutturn': 32030},
  {'publishTime': '2024-05-04T18:30:00Z',
   'startTime': '2024-05-04T18:00:00Z',
   'settlementDate': '2024-05-04',
   'settlementPeriod': 39,
   'initialTransmissionSystemDemandOutturn': 29912},
  {'publishTime': '2024-05-05T20:00:00Z',
   'startTime': '2024-05-05T19:30:00Z',
   'settlementDate': '2024-05-05',
   'settlementPeriod': 42,
   'initialTransmissio

In [301]:
demand_peak.head(1)

,settlementPeriod,publishTime,initialTransmissionSystemDemandOutturn,settlementDate,startTime
0,39,2024-05-01T18:30:00Z,32354,2024-05-01,2024-05-01T18:00:00Z


In [302]:
demand_peak_META

,datasets
0,ITSDO


In [303]:
# get a list of columns to compare
all_columns = demand_peak_API.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(demand_peak,
                                                         demand_peak_API,
                                                         columns_to_compare)
are_equal

True

#### ENDPOINT 11:  get_datasets_indod_publishdatetimefrom_publishdatetimefrom_publishdatetimeto


In [304]:
datasets_indod,datasets_indod_META = get_datasets_indod_publishdatetimefrom_publishdatetimefrom_publishdatetimeto('2024-05-01T00:00:00Z', '2024-05-02T00:00:00Z', json)

# Get data directly from API
datasets_indod_API,datasets_indod_API_META = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/datasets/INDOD?publishDateTimeFrom=2024-05-01T00:00:00Z&publishDateTimeTo=2024-05-02T00:00:00Z&format=json')

In [305]:
datasets_indod_json = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/datasets/INDOD?publishDateTimeFrom=2024-05-01T00:00:00Z&publishDateTimeTo=2024-05-02T00:00:00Z&format=json')
datasets_indod_json

{'data': [{'dataset': 'INDOD',
   'publishTime': '2024-05-01T23:15:00Z',
   'settlementDate': '2024-05-01',
   'demand': 625461}]}

In [306]:
datasets_indod

,demand,publishTime,dataset,settlementDate
0,625461,2024-05-01T23:15:00Z,INDOD,2024-05-01


In [307]:
datasets_indod_META

'No metadata provided'

In [308]:
# get a list of columns to compare
all_columns = datasets_indod_API.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(datasets_indod,
                                                         datasets_indod_API,
                                                         columns_to_compare)
are_equal

True

#### ENDPOINT 12: get_datasets_boalf_stream_from_from_to_to


In [309]:
# Get data directly from API
datasets_boalf_stream, datasets_boalf_stream_META = get_datasets_boalf_stream_from_from_to_to('2024-05-01T00:00:00Z', '2024-05-02T00:00:00Z', 1, 2, None)

datasets_boalf_stream_API,datasets_boalf_stream_API_META = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/datasets/BOALF/stream?from=2024-05-01T00:00:00Z&to=2024-05-02T00:00:00Z&settlementPeriodFrom=1&settlementPeriodTo=2')

In [310]:
datasets_boalf_stream_json = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/datasets/BOALF/stream?from=2024-05-01T00:00:00Z&to=2024-05-02T00:00:00Z&settlementPeriodFrom=1&settlementPeriodTo=2')
datasets_boalf_stream_json

[{'dataset': 'BOALF',
  'settlementDate': '2024-05-02',
  'settlementPeriodFrom': 2,
  'settlementPeriodTo': 3,
  'timeFrom': '2024-05-01T23:59:00Z',
  'timeTo': '2024-05-02T00:00:00Z',
  'levelFrom': 50,
  'levelTo': 100,
  'acceptanceNumber': 6182,
  'acceptanceTime': '2024-05-01T23:22:00Z',
  'deemedBoFlag': False,
  'soFlag': False,
  'amendmentFlag': 'ORI',
  'storFlag': False,
  'rrFlag': False,
  'nationalGridBmUnit': 'SGRWO-1',
  'bmUnit': 'T_SGRWO-1'},
 {'dataset': 'BOALF',
  'settlementDate': '2024-05-02',
  'settlementPeriodFrom': 2,
  'settlementPeriodTo': 3,
  'timeFrom': '2024-05-01T23:59:00Z',
  'timeTo': '2024-05-02T00:00:00Z',
  'levelFrom': 50,
  'levelTo': 100,
  'acceptanceNumber': 7836,
  'acceptanceTime': '2024-05-01T23:22:00Z',
  'deemedBoFlag': False,
  'soFlag': False,
  'amendmentFlag': 'ORI',
  'storFlag': False,
  'rrFlag': False,
  'nationalGridBmUnit': 'SGRWO-6',
  'bmUnit': 'T_SGRWO-6'},
 {'dataset': 'BOALF',
  'settlementDate': '2024-05-02',
  'settlemen

In [311]:
datasets_boalf_stream.head(1)

,dataset,settlementDate,settlementPeriodFrom,settlementPeriodTo,timeFrom,timeTo,levelFrom,levelTo,acceptanceNumber,acceptanceTime,deemedBoFlag,soFlag,amendmentFlag,storFlag,rrFlag,nationalGridBmUnit,bmUnit
0,BOALF,2024-05-02,2,3,2024-05-01T23:59:00Z,2024-05-02T00:00:00Z,50,100,6182,2024-05-01T23:22:00Z,False,False,ORI,False,False,SGRWO-1,T_SGRWO-1


In [312]:
datasets_boalf_stream_META

'No metadata provided'

In [313]:
# get a list of columns to compare
all_columns = datasets_boalf_stream_API.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(datasets_boalf_stream,
                                                         datasets_boalf_stream_API,
                                                         columns_to_compare)
are_equal

True

#### ENDPOINT 13: 'get_demand_outturn',


In [314]:
demand_outturn_daily,demand_outturn_daily_META = get_demand_outturn_daily('2024-05-01', '2024-05-07', json)

# Get data directly from API
demand_outturn_daily_API, demand_outturn_daily_API_META = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/demand/outturn/daily?settlementDateFrom=2024-05-01&settlementDateTo=2024-05-07&format=json')

In [315]:
demand_outturn_daily_json = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/demand/outturn/daily?settlementDateFrom=2024-05-01&settlementDateTo=2024-05-07&format=json')
demand_outturn_daily_json

{'metadata': {'datasets': ['INDOD']},
 'data': [{'publishTime': '2024-05-07T23:15:00Z',
   'settlementDate': '2024-05-07',
   'demand': 588794},
  {'publishTime': '2024-05-06T23:15:00Z',
   'settlementDate': '2024-05-06',
   'demand': 553419},
  {'publishTime': '2024-05-05T23:15:00Z',
   'settlementDate': '2024-05-05',
   'demand': 526224},
  {'publishTime': '2024-05-04T23:15:00Z',
   'settlementDate': '2024-05-04',
   'demand': 533399},
  {'publishTime': '2024-05-03T23:15:00Z',
   'settlementDate': '2024-05-03',
   'demand': 628864},
  {'publishTime': '2024-05-02T23:15:00Z',
   'settlementDate': '2024-05-02',
   'demand': 604295},
  {'publishTime': '2024-05-01T23:15:00Z',
   'settlementDate': '2024-05-01',
   'demand': 625461}]}

In [316]:
demand_outturn_daily.head(1)

,demand,publishTime,settlementDate
0,588794,2024-05-07T23:15:00Z,2024-05-07


In [317]:
demand_outturn_daily_META

,datasets
0,INDOD


In [318]:
# get a list of columns to compare
all_columns = demand_outturn_daily_API.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(demand_outturn_daily,
                                                         demand_outturn_daily_API,
                                                         columns_to_compare)
are_equal

True

#### ENDPOINT 14: get_datasets_tsdfw


In [319]:
#datasets_tsdfw = get_datasets_tsdfw('2024-05-01', '2024-05-02', json)

#datasets_tsdfw_API = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/datasets/TSDFW?publishDateTimeFrom=2024-05-01&publishDateTimeTo=2024-05-02&format=json')

# get a list of columns to compare
#all_columns = datasets_tsdfw_API.columns
#columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
#are_equal, diff_rows_mockdata, diff_rows_list = df_equal(datasets_tsdfw, datasets_tsdfw_API,columns_to_compare)
#are_equal

print('no data')

no data


#### ENDPOINT 15: get_forecast_generation_wind_peak


In [320]:
forecast_generation_wind_peak, forecast_generation_wind_peak_META = get_forecast_generation_wind_peak('2024-05-01', '2024-05-07', json)

# Get data directly from API
forecast_generation_wind_peak_API,forecast_generation_wind_peak_API_META = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/forecast/generation/wind/peak?from=2024-05-01&to=2024-05-07&format=json')

In [321]:
forecast_generation_wind_peak_json = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/forecast/generation/wind/peak?from=2024-05-01&to=2024-05-07&format=json')
forecast_generation_wind_peak_json

{'metadata': {'datasets': ['WINDFOR']},
 'data': [{'publishTime': '2024-05-01T23:30:00Z',
   'startTime': '2024-05-01T22:00:00Z',
   'settlementDate': '2024-05-01',
   'settlementPeriod': 47,
   'generation': 6525},
  {'publishTime': '2024-05-01T23:30:00Z',
   'startTime': '2024-05-02T14:00:00Z',
   'settlementDate': '2024-05-02',
   'settlementPeriod': 31,
   'generation': 10508},
  {'publishTime': '2024-05-02T23:30:00Z',
   'startTime': '2024-05-02T23:00:00Z',
   'settlementDate': '2024-05-03',
   'settlementPeriod': 1,
   'generation': 6970},
  {'publishTime': '2024-05-03T23:30:00Z',
   'startTime': '2024-05-03T23:00:00Z',
   'settlementDate': '2024-05-04',
   'settlementPeriod': 1,
   'generation': 6431},
  {'publishTime': '2024-05-05T23:30:00Z',
   'startTime': '2024-05-05T21:00:00Z',
   'settlementDate': '2024-05-05',
   'settlementPeriod': 45,
   'generation': 2008},
  {'publishTime': '2024-05-05T23:30:00Z',
   'startTime': '2024-05-06T19:00:00Z',
   'settlementDate': '2024-05-0

In [322]:
forecast_generation_wind_peak.head(1)

,settlementPeriod,publishTime,settlementDate,startTime,generation
0,47,2024-05-01T23:30:00Z,2024-05-01,2024-05-01T22:00:00Z,6525


In [323]:
forecast_generation_wind_peak_META

,datasets
0,WINDFOR


In [324]:
# get a list of columns to compare
all_columns = forecast_generation_wind_peak_API.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(forecast_generation_wind_peak,
                                                         forecast_generation_wind_peak_API,
                                                         columns_to_compare)
are_equal

True

#### ENDPOINT 16: get_datasets_nto_stream_from_from_to_to


In [325]:
datasets_nto_stream, datasets_nto_stream_META = get_datasets_nto_stream_from_from_to_to('2024-05-01T00:00:00Z', '2024-05-02T00:00:00Z', 2, 5, None)

# Get data directly from API
datasets_nto_stream_API, datasets_nto_stream_Meta= fetch_data('https://data.elexon.co.uk/bmrs/api/v1/datasets/NTO/stream?from=2024-05-01T00:00:00Z&to=2024-05-02T00:00:00Z&settlementPeriodFrom=2&settlementPeriodTo=5')

In [326]:
datasets_nto_stream_json= fetch_json('https://data.elexon.co.uk/bmrs/api/v1/datasets/NTO/stream?from=2024-05-01T00:00:00Z&to=2024-05-02T00:00:00Z&settlementPeriodFrom=2&settlementPeriodTo=5')
datasets_nto_stream_json

[{'dataset': 'NTO',
  'settlementDate': '2024-05-02',
  'settlementPeriod': 3,
  'time': '2024-05-02T00:20:00Z',
  'notice': 2,
  'nationalGridBmUnit': 'FOYE-1',
  'bmUnit': 'T_FOYE-1'},
 {'dataset': 'NTO',
  'settlementDate': '2024-05-02',
  'settlementPeriod': 1,
  'time': '2024-05-01T23:13:00Z',
  'notice': 2,
  'nationalGridBmUnit': 'FOYE-1',
  'bmUnit': 'T_FOYE-1'},
 {'dataset': 'NTO',
  'settlementDate': '2024-05-02',
  'settlementPeriod': 1,
  'time': '2024-05-01T23:13:00Z',
  'notice': 2,
  'nationalGridBmUnit': 'FOYE-2',
  'bmUnit': 'T_FOYE-2'},
 {'dataset': 'NTO',
  'settlementDate': '2024-05-01',
  'settlementPeriod': 40,
  'time': '2024-05-01T18:33:00Z',
  'notice': 2,
  'nationalGridBmUnit': 'PEMB-11',
  'bmUnit': 'T_PEMB-11'},
 {'dataset': 'NTO',
  'settlementDate': '2024-05-01',
  'settlementPeriod': 36,
  'time': '2024-05-01T16:58:00Z',
  'notice': 2,
  'nationalGridBmUnit': 'PEMB-11',
  'bmUnit': 'T_PEMB-11'},
 {'dataset': 'NTO',
  'settlementDate': '2024-05-01',
  'se

In [327]:
datasets_nto_stream.head(1)

,dataset,settlementDate,settlementPeriod,time,notice,nationalGridBmUnit,bmUnit
0,NTO,2024-05-02,3,2024-05-02T00:20:00Z,2,FOYE-1,T_FOYE-1


In [328]:
datasets_nto_stream_META

'No metadata provided'

In [329]:
# get a list of columns to compare
all_columns = datasets_nto_stream_API.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(datasets_nto_stream,
                                                         datasets_nto_stream_API,
                                                         columns_to_compare)
are_equal

True

#### ENDPOINT 17: get_forecast_surplus_weekly_history_publishtime_publishtime


In [330]:
forecast_surplus_weekly, forecast_surplus_weekly_META = get_forecast_surplus_weekly_history_publishtime_publishtime('2024-05-01T00:00:00Z', '52W', json)

# Get data directly from API
forecast_surplus_weekly_API,forecast_surplus_weekly_API_META = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/forecast/surplus/weekly/history?publishTime=2024-05-01T00:00:00Z&range=52W&format=json')

In [331]:
forecast_surplus_weekly_json = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/forecast/surplus/weekly/history?publishTime=2024-05-01T00:00:00Z&range=52W&format=json')
forecast_surplus_weekly_json

{'metadata': {'datasets': ['OCNMFW']},
 'data': [{'week': 20,
   'year': 2024,
   'weekStartDate': '2024-05-13',
   'publishTime': '2024-04-30T23:17:00Z',
   'surplus': 12950},
  {'week': 21,
   'year': 2024,
   'weekStartDate': '2024-05-20',
   'publishTime': '2024-04-30T23:17:00Z',
   'surplus': 9954},
  {'week': 22,
   'year': 2024,
   'weekStartDate': '2024-05-27',
   'publishTime': '2024-04-30T23:17:00Z',
   'surplus': 12491},
  {'week': 23,
   'year': 2024,
   'weekStartDate': '2024-06-03',
   'publishTime': '2024-04-30T23:17:00Z',
   'surplus': 9972},
  {'week': 24,
   'year': 2024,
   'weekStartDate': '2024-06-10',
   'publishTime': '2024-04-30T23:17:00Z',
   'surplus': 10241},
  {'week': 25,
   'year': 2024,
   'weekStartDate': '2024-06-17',
   'publishTime': '2024-04-30T23:17:00Z',
   'surplus': 10654},
  {'week': 26,
   'year': 2024,
   'weekStartDate': '2024-06-24',
   'publishTime': '2024-04-30T23:17:00Z',
   'surplus': 13375},
  {'week': 27,
   'year': 2024,
   'weekStart

In [332]:
forecast_surplus_weekly.head(1)

,surplus,year,publishTime,week,weekStartDate
0,12950,2024,2024-04-30T23:17:00Z,20,2024-05-13


In [333]:
forecast_surplus_weekly_META

,datasets
0,OCNMFW


In [334]:
# get a list of columns to compare
all_columns = forecast_surplus_weekly_API.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(forecast_surplus_weekly,
                                                         forecast_surplus_weekly_API,
                                                         columns_to_compare)
are_equal

True

#### ENDPOINT 18: get_datasets_ntb_stream_from_from_to_to


In [335]:
datasets_ntb_stream, datasets_ntb_stream_META = get_datasets_ntb_stream_from_from_to_to('2024-05-01T00:00:00Z', '2024-05-02T00:00:00Z', 2, 5, None)

# Get data directly from API
datasets_ntb_stream_API,datasets_ntb_stream_API_META = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/datasets/NTB/stream?from=2024-05-01T00:00:00Z&to=2024-05-02T00:00:00Z&settlementPeriodFrom=2&settlementPeriodTo=5')

In [336]:
datasets_ntb_stream_json = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/datasets/NTB/stream?from=2024-05-01T00:00:00Z&to=2024-05-02T00:00:00Z&settlementPeriodFrom=2&settlementPeriodTo=5')
datasets_ntb_stream_json

[{'dataset': 'NTB',
  'settlementDate': '2024-05-02',
  'settlementPeriod': 3,
  'time': '2024-05-02T00:21:00Z',
  'notice': 2,
  'nationalGridBmUnit': 'FOYE-1',
  'bmUnit': 'T_FOYE-1'},
 {'dataset': 'NTB',
  'settlementDate': '2024-05-02',
  'settlementPeriod': 1,
  'time': '2024-05-01T23:13:00Z',
  'notice': 2,
  'nationalGridBmUnit': 'FOYE-1',
  'bmUnit': 'T_FOYE-1'},
 {'dataset': 'NTB',
  'settlementDate': '2024-05-02',
  'settlementPeriod': 1,
  'time': '2024-05-01T23:13:00Z',
  'notice': 2,
  'nationalGridBmUnit': 'FOYE-2',
  'bmUnit': 'T_FOYE-2'},
 {'dataset': 'NTB',
  'settlementDate': '2024-05-01',
  'settlementPeriod': 40,
  'time': '2024-05-01T18:33:00Z',
  'notice': 2,
  'nationalGridBmUnit': 'PEMB-11',
  'bmUnit': 'T_PEMB-11'},
 {'dataset': 'NTB',
  'settlementDate': '2024-05-01',
  'settlementPeriod': 36,
  'time': '2024-05-01T16:58:00Z',
  'notice': 2,
  'nationalGridBmUnit': 'PEMB-11',
  'bmUnit': 'T_PEMB-11'},
 {'dataset': 'NTB',
  'settlementDate': '2024-05-01',
  'se

In [337]:
datasets_ntb_stream.head(1)

,dataset,settlementDate,settlementPeriod,time,notice,nationalGridBmUnit,bmUnit
0,NTB,2024-05-02,3,2024-05-02T00:21:00Z,2,FOYE-1,T_FOYE-1


In [338]:
datasets_ntb_stream_META

'No metadata provided'

In [339]:
# get a list of columns to compare
all_columns = datasets_ntb_stream_API.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(datasets_ntb_stream,
                                                         datasets_ntb_stream_API,
                                                         columns_to_compare)
are_equal

True

#### ENDPOINT 19: get_lolpdrm_forecast_evolution


In [340]:
# get_lolpdrm_forecast_evolution(json)

print('# This endpoint has been moved to forecast/system/loss-of-load')

# This endpoint has been moved to forecast/system/loss-of-load


#### ENDPOINT 20: get_datasets_nou2t3yw


In [341]:
datasets_nou2t3yw,datasets_nou2t3yw_META= get_datasets_nou2t3yw('2024-05-01', None, None, None, None, json)

# Get data directly from API
datasets_nou2t3yw_API, datasets_nou2t3yw_API_META = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/datasets/NOU2T3YW?publishDate=2024-05-01&format=json')

In [342]:
datasets_nou2t3yw_json = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/datasets/NOU2T3YW?publishDate=2024-05-01&format=json')
datasets_nou2t3yw_json

{'data': [{'dataset': 'NOU2T3YW',
   'publishTime': '2024-05-01T23:00:00Z',
   'systemZone': None,
   'calendarWeekNumber': 20,
   'year': 2024,
   'outputUsable': 45967},
  {'dataset': 'NOU2T3YW',
   'publishTime': '2024-05-01T23:00:00Z',
   'systemZone': None,
   'calendarWeekNumber': 21,
   'year': 2024,
   'outputUsable': 42910},
  {'dataset': 'NOU2T3YW',
   'publishTime': '2024-05-01T23:00:00Z',
   'systemZone': None,
   'calendarWeekNumber': 22,
   'year': 2024,
   'outputUsable': 43468},
  {'dataset': 'NOU2T3YW',
   'publishTime': '2024-05-01T23:00:00Z',
   'systemZone': None,
   'calendarWeekNumber': 23,
   'year': 2024,
   'outputUsable': 41380},
  {'dataset': 'NOU2T3YW',
   'publishTime': '2024-05-01T23:00:00Z',
   'systemZone': None,
   'calendarWeekNumber': 24,
   'year': 2024,
   'outputUsable': 41487},
  {'dataset': 'NOU2T3YW',
   'publishTime': '2024-05-01T23:00:00Z',
   'systemZone': None,
   'calendarWeekNumber': 25,
   'year': 2024,
   'outputUsable': 41775},
  {'data

In [343]:
datasets_nou2t3yw.head(1)

,systemZone,year,calendarWeekNumber,publishTime,dataset,outputUsable
0,None,2024,20,2024-05-01T23:00:00Z,NOU2T3YW,45967


In [344]:
datasets_nou2t3yw_META

'No metadata provided'

In [345]:
# get a list of columns to compare
all_columns = datasets_nou2t3yw_API.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(datasets_nou2t3yw,
                                                         datasets_nou2t3yw_API,
                                                         columns_to_compare)
are_equal

True

#### ENDPOINT 21: datasets_tsdfw

In [346]:
datasets_tsdfw, datasets_tsdfw_META= get_datasets_tsdfw('2024-05-01T00:00:00Z', '2024-05-07T23:59:59Z', 'json')

# Get data directly from API
datasets_tsdfw_API,datasets_tsdfw_API_META = fetch_data('https://data.elexon.co.uk/bmrs/api/v1/datasets/TSDFW?publishDateTimeFrom=2024-05-01T00:00:00Z&publishDateTimeTo=2024-05-07T23:59:59Z&format=json')

In [347]:
datasets_tsdfw_json = fetch_json('https://data.elexon.co.uk/bmrs/api/v1/datasets/TSDFW?publishDateTimeFrom=2024-05-01T00:00:00Z&publishDateTimeTo=2024-05-07T23:59:59Z&format=json')
datasets_tsdfw_json

{'data': [{'dataset': 'TSDFW',
   'publishTime': '2024-05-02T13:45:00Z',
   'year': 2024,
   'week': 20,
   'demand': 29780},
  {'dataset': 'TSDFW',
   'publishTime': '2024-05-02T13:45:00Z',
   'year': 2024,
   'week': 21,
   'demand': 29520},
  {'dataset': 'TSDFW',
   'publishTime': '2024-05-02T13:45:00Z',
   'year': 2024,
   'week': 22,
   'demand': 28140},
  {'dataset': 'TSDFW',
   'publishTime': '2024-05-02T13:45:00Z',
   'year': 2024,
   'week': 23,
   'demand': 28150},
  {'dataset': 'TSDFW',
   'publishTime': '2024-05-02T13:45:00Z',
   'year': 2024,
   'week': 24,
   'demand': 27590},
  {'dataset': 'TSDFW',
   'publishTime': '2024-05-02T13:45:00Z',
   'year': 2024,
   'week': 25,
   'demand': 27920},
  {'dataset': 'TSDFW',
   'publishTime': '2024-05-02T13:45:00Z',
   'year': 2024,
   'week': 26,
   'demand': 27590},
  {'dataset': 'TSDFW',
   'publishTime': '2024-05-02T13:45:00Z',
   'year': 2024,
   'week': 27,
   'demand': 28490},
  {'dataset': 'TSDFW',
   'publishTime': '2024-0

In [348]:
datasets_tsdfw.head(1)

,year,publishTime,demand,dataset,week
0,2024,2024-05-02T13:45:00Z,29780,TSDFW,20


In [349]:
datasets_tsdfw_META

'No metadata provided'

In [350]:
# get a list of columns to compare
all_columns = datasets_tsdfw_API.columns
columns_to_compare = [colums for colums in all_columns]

# Call function to check is dataframes are equal
are_equal, diff_rows_mockdata, diff_rows_list = df_equal(datasets_tsdfw,
                                                         datasets_tsdfw_API,
                                                         columns_to_compare)
are_equal

True